# Proyecto: Zona de Cinéfilos - SRP

# ETL

### Carga de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import openpyxl
import json
import os
import webbrowser
import platform
import requests
import pymongo
import dash
import pymongo
import pyarrow as pa
import pyarrow.parquet as pq
import nltk
import nltk.corpus

from tabulate import tabulate
from pprintjson import pprintjson as ppjson
from pymongo import MongoClient
from datetime import datetime
from fastapi import FastAPI
from dash.dependencies import Input, Output
from dash import dcc
from dash import html

# Remueve los límites para el número de columnas desplegadas 
# (en su defecto muestra un separador '...')
# ==============================================================================

pd.set_option("display.max_columns", None)
pd.set_option('format.precision', 2)


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from rake_nltk import Rake
from gotstopwords import gotstopwords

## Carga de datos

In [2]:
df_movies_or = pd.read_csv('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/rawdata/movies_dataset.csv', sep = ',')
df_movies_or.head(1)

C:\Users\Estudio\AppData\Local\Temp\ipykernel_13724\580784504.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_or = pd.read_csv('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/rawdata/movies_dataset.csv', sep = ',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [46]:
df_work = df_movies_or

In [3]:
df_credits = pd.read_csv('credits.csv')
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:

df_credits.shape


(45476, 3)

In [5]:
df_credits.cast.loc[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

## Chequea nulos y faltantes

In [6]:
#Vemos si hay valores faltantes/nulos

df_movies_or.isna().sum()


adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

## Tratamiento categoría 'adult'

Se chequean las películas para adultos y se eliminan.

Se elimna la columna 'adult'

In [7]:
adult = df_movies_or['adult'].unique().tolist()
adult

['False',
 'True',
 ' - Written by Ørnås',
 ' Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.',
 ' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.']

In [8]:
# Filtrar y eliminar películas y columna 'adult'
# *********************************************
# Preprocesa columna 'adult', elimina películas
# de adultos.

def filtrar_adult(df):
    """
    Función elimina la columna 'adult' y las películas
    para adultos.
    
    Recibe  : Un df, chequea la columna 'adult'
    Retorna : El df filtrado
    
    """
    # Filtra los valores 'True' en la columna 'adult'
    df_fil = df[df['adult'] != "True"].copy()

    # Verifica que no quedaron valores 'True' en la columna filtrada
    verifica = "True" not in df_fil['adult'].values

    # Limpiado los datos, se elimina la columna 'adult'
    df_fil.drop('adult', axis = 1, inplace = True)

    return df_fil, verifica

df_movies_or, verificado = filtrar_adult(df_movies_or)
print(df_movies_or.head(2))
print("No hay valores 'True':", verificado)

                               belongs_to_collection    budget  \
0  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                NaN  65000000   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   

                               homepage    id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
1                                   NaN  8844  tt0113497                en   

  original_title                                           overview  \
0      Toy Story  Led by Woody, Andy's toys live happily in his ...   
1        Jumanji  When siblings Judy and Peter discover an encha...   

  popularity                       poster_path  \
0  21.946943  /rhIRbceoE9lR4veEXuwCC2wARtG.jpg   
1  17.015539  /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg   

                                

In [ ]:
# Comprueba que se efectuaron los cambios
# !!!! DEBE GENERAR ERROR.

por_nombre = df_movies_or[df_movies_or['adult'] == 'True' ]
por_nombre

# Insight : Debe generar error luego de correr el proceso de
# eliminación de películas para adultos y la columna 'adult'.

In [10]:
# Columnas que van quedando por procesar

df_movies_or.columns

Index(['belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

### Preprocesamiento ETL - Primera aproximación

In [11]:
# Preprocesamiento de datos - ETL 
# *******************************
# Función que reune las tareas operativas descritas 
# en el enunciado, incluye: Rellenar nulos en 'revenue y
# 'budget', dropea valores de columna fecha, chequea
# formato corto en columna fecha (AAAA-mm-dd), crea columna año
# también crea mes, dia, y día de la semana, elimina duplicados.

def preproceso_df(df):
    """
    Recibe  : Un df y aplica reglas indicadas en el enunciado
    verifica, elimina o reemplaza nulos, formato, rellena con 
    0.
    Retorna : df preprocesado.
    """
    
    # Rellena valores nulos de las columnas 'revenue' y 'budget' con 0
    df['revenue'].fillna(0, inplace=True)
    df['budget'].fillna(0, inplace=True)

    # Elimina valores nulos del campo 'release_date'
    df.dropna(subset=['release_date'], inplace=True)

    # Verifica y convierte el formato de 'release_date' a fecha corta (AAAA-mm-dd)
    if df['release_date'].dtype == 'object':
        df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', infer_datetime_format=True)
        df['release_date'] = df['release_date'].dt.strftime('%Y-%m-%d')

    # Crea columna 'release_year' extrayendo el año de 'release_date'
    df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce', infer_datetime_format=True).dt.year
    df['release_year'].fillna("Sin dato", inplace=True)

    # Extraer el número de mes y día de 'release_date'
    df['release_month'] = pd.to_datetime(df['release_date'], errors='coerce', infer_datetime_format=True).dt.month
    df['release_day']   = pd.to_datetime(df['release_date'], errors='coerce', infer_datetime_format=True).dt.day

    # Convertir el día de la fecha a equivalente del día de la semana
    df['release_weekday'] = pd.to_datetime(df['release_date'], errors='coerce', infer_datetime_format=True).dt.weekday + 1

    # Eliminar duplicados
    df.drop_duplicates(inplace=True)

    # Corregir el error ValueError al procesar las fechas
    df['release_day'].fillna("Sin dato", inplace=True)
    df['release_month'].fillna("Sin dato", inplace=True)
    df['release_weekday'].fillna("Sin dato", inplace=True)

    # Convertir 'revenue' y 'budget' a tipo numérico
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
    df['budget']  = pd.to_numeric(df['budget'], errors='coerce')

    # Crear la columna 'return' con el cociente entre 'revenue' y 'budget'
    df['return'] = df['revenue'] / df['budget']
    df['return'].fillna(0, inplace=True)

    # Eliminar las columnas 'video', 'original_title', 'poster_path' y 'homepage'
    df.drop(columns = ['video', 'original_title', 'poster_path', 'homepage'], inplace=True)

    # Se eliminan los duplicados

    df.drop_duplicates(inplace=True)
    
    # Genera una muestra de 3 registros usando sample
    muestra = df.sample(n=3)

    return df, muestra

In [12]:
df_movies_or, muestra = preproceso_df(df_movies_or)

In [13]:
def reemplazar_comillas(df):
    df = df.replace("'", '"', regex=True)
    return df

df_movies_or = reemplazar_comillas(df_movies_or)
df_movies_or.head(1)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return
0,"{""id"": 10194, ""name"": ""Toy Story Collection"", ...",30000000.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""...",862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-10-30,373554033.0,81.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801


In [116]:
df_movies_or.sample(1)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return
33702,NaN,4500000.0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",55765,tt0891488,ko,"An eccentric but faithful mafia, Chi-sung is s...",1.602638,[],"[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",2006-10-19,0.0,126.0,"[{""iso_639_1"": ""ko"", ""name"": ""한국어/조선말""}]",Released,NaN,Righteous Ties,6.5,12.0,2006.0,10.0,19.0,4.0,0.0


In [14]:
df_movies_or.shape


(45358, 24)

In [15]:
# Compobamos que no existan títulos de la categoría 'adult'
por_nombre = df_movies_or[ df_movies_or['title'] == 'Erotic Nights of the Living Dead' ]
por_nombre

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return


In [16]:
# df para pruebas, objetivo dividir el campo en los generos respectivos.
df_genres = pd.read_excel('D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\dfwork_200623.xlsx', sheet_name='Sheet1', usecols=['genres'] )

In [17]:
df_genres.head(1)

,genres
0,"{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""n..."


In [18]:
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   genres  45379 non-null  object
dtypes: object(1)
memory usage: 354.7+ KB


In [19]:
df_movies_or.isna().sum()

belongs_to_collection    40867
budget                       3
genres                       0
id                           0
imdb_id                     14
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 3
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24970
title                        3
vote_average                 3
vote_count                   3
release_year                 0
release_month                0
release_day                  0
release_weekday              0
return                       0
dtype: int64

In [20]:
# Función que corrige valores nulos
# *********************************
# Motor de reglas que recopila cambios en 
# 11 columnas, verifica tipos, agrega texto, 
# valor 0 o datos json.
# Recibe  : El df
# Retorna : El df procesado

def corregir_valores_nulos(df):
    # Regla 1: Chequear y reemplazar valores nulos en "belongs_to_collection"
    if 'belongs_to_collection' in df.columns:
        df['belongs_to_collection'] = df['belongs_to_collection'].fillna('{"id": 999999, "name": "Sin dato"}')
    
    # Reindexar el DataFrame después de aplicar la regla 1
    df = df.reset_index(drop=True)
    
    # Regla 2: Reemplazar valores nulos en "budget" con 0
    if 'budget' in df.columns:
        df['budget'] = df['budget'].fillna(0)
    
    # Reindexar el DataFrame después de aplicar la regla 2
    df = df.reset_index(drop=True)
    
    # Regla 3: Reemplazar valores nulos en "idmdb_id" con "tt0000000"
    if 'imdb_id' in df.columns:
        df['imdb_id'] = df['imdb_id'].fillna('tt0000000')
    
    # Reindexar el DataFrame después de aplicar la regla 3
    df = df.reset_index(drop=True)
    
    # Regla 4: Reemplazar valores nulos o no válidos en "original_language" con "xx"
    if 'original_language' in df.columns:
        df['original_language'] = df['original_language'].fillna('xx')
        df['original_language'] = df['original_language'].apply(lambda x: x[:2].lower())
    
    # Reindexar el DataFrame después de aplicar la regla 4
    df = df.reset_index(drop=True)
    
    # Regla 5: Reemplazar valores nulos en "overview" con "Sin dato"
    if 'overview' in df.columns:
        df['overview'] = df['overview'].fillna('Sin dato')
    
    # Reindexar el DataFrame después de aplicar la regla 5
    df = df.reset_index(drop=True)
    
    # Regla 6: Reemplazar valores nulos o no válidos en "popularity" con 0
    if 'popularity' in df.columns:
        df['popularity'] = df['popularity'].fillna(0)
    
    # Reindexar el DataFrame después de aplicar la regla 6
    df = df.reset_index(drop=True)
    
    # Regla 7: Eliminar registros con valores nulos o no válidos en "release_date"
    if 'release_date' in df.columns:
        df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
        df = df.dropna(subset=['release_date'])
    
    # Reindexar el DataFrame después de aplicar la regla 7
    df = df.reset_index(drop=True)
    
    # Regla 8: Reemplazar valores nulos o no válidos en "runtime" con 0
    if 'runtime' in df.columns:
        df['runtime'] = df['runtime'].fillna(0)
    
    # Reindexar el DataFrame después de aplicar la regla 8
    df = df.reset_index(drop=True)
    
    # Regla 9: Chequear y reemplazar valores nulos en "spoken_languages"
    if 'spoken_languages' in df.columns:
        df['spoken_languages'] = df['spoken_languages'].fillna('{"iso_639_1": "xx", "name": "No Language"}')
        df['spoken_languages'] = df['spoken_languages'].apply(lambda x: x.replace('""', '"No Language"'))
    
    # Reindexar el DataFrame después de aplicar la regla 9
    df = df.reset_index(drop=True)
    
    # Regla 10: Reemplazar valores nulos o no válidos en "status" con "Sin dato"
    if 'status' in df.columns:
        df['status'] = df['status'].fillna('Sin dato')
    
    # Reindexar el DataFrame después de aplicar la regla 10
    df = df.reset_index(drop=True)
    
    # Regla 11: Reemplazar valores nulos en "tagline" con "Sin dato"
    if 'tagline' in df.columns:
        df['tagline'] = df['tagline'].fillna('Sin dato')
    
    # Reindexar el DataFrame después de aplicar la regla 11
    df = df.reset_index(drop=True)
    
    # Regla 12: Eliminar registros con valores nulos o no válidos en "title"
    if 'title' in df.columns:
        df = df.dropna(subset=['title'])
    
    # Reindexar el DataFrame después de aplicar la regla 12
    df = df.reset_index(drop=True)
    
    # Regla 13: Reemplazar valores nulos o no válidos en "vote_average" con 0
    if 'vote_average' in df.columns:
        df['vote_average'] = df['vote_average'].fillna(0)
    
    # Reindexar el DataFrame después de aplicar la regla 13
    df = df.reset_index(drop=True)
    
    # Regla 14: Reemplazar valores nulos o no válidos en "vote_count" con 0
    if 'vote_count' in df.columns:
        df['vote_count'] = df['vote_count'].fillna(0)
    
    # Reindexar el DataFrame después de aplicar la regla 14
    df = df.reset_index(drop=True)
    
    # Regla 15: Generar reporte de cambios
    cambios = df.isna().sum()
    reporte_cambios = pd.DataFrame(cambios, columns=['Valores Nulos'])
    reporte_cambios['Valores Actualizados'] = len(df) - cambios
    reporte_cambios.columns = ['Valores Nulos', 'Valores Actualizados']
    
    return df, reporte_cambios


In [21]:
df_movies_or, reporte_cambios  = corregir_valores_nulos(df_movies_or)

In [22]:
reporte_cambios 

,Valores Nulos,Valores Actualizados
belongs_to_collection,0,45355
budget,0,45355
genres,0,45355
id,0,45355
imdb_id,0,45355
original_language,0,45355
overview,0,45355
popularity,0,45355
production_companies,0,45355
production_countries,0,45355


In [23]:
# Muestra cambios
df_movies_or.belongs_to_collection.iloc[12000:12015]

12000                   {"id": 999999, "name": "Sin dato"}
12001                   {"id": 999999, "name": "Sin dato"}
12002                   {"id": 999999, "name": "Sin dato"}
12003                   {"id": 999999, "name": "Sin dato"}
12004                   {"id": 999999, "name": "Sin dato"}
12005    {"id": 228887, "name": "Daddy Day Camp Collect...
12006                   {"id": 999999, "name": "Sin dato"}
12007                   {"id": 999999, "name": "Sin dato"}
12008                   {"id": 999999, "name": "Sin dato"}
12009                   {"id": 999999, "name": "Sin dato"}
12010                   {"id": 999999, "name": "Sin dato"}
12011    {"id": 126209, "name": "Halloween (Rob Zombie ...
12012                   {"id": 999999, "name": "Sin dato"}
12013    {"id": 138908, "name": "2 Days In... Collectio...
12014                   {"id": 999999, "name": "Sin dato"}
Name: belongs_to_collection, dtype: object

In [24]:
# df_movies_or.belongs_to_collection.iloc[12000:12015]

In [125]:
# Verificar tipo de datos de las columnas
print(df_movies_or.dtypes)

belongs_to_collection            object
budget                          float64
genres                           object
id                               object
imdb_id                          object
original_language                object
overview                         object
popularity                       object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                    float64
vote_count                      float64
release_year                     object
release_month                    object
release_day                      object
release_weekday                  object
return                          float64
dtype: object


In [25]:
df_movies_or.release_date.iloc[1200:1215]

1200   1967-12-21
1201   1958-04-23
1202   1990-02-21
1203   1957-10-02
1204   1963-02-14
1205   1974-06-20
1206   1951-09-17
1207   1948-01-24
1208   1987-12-01
1209   1933-11-17
1210   1985-10-11
1211   1980-05-22
1212   1986-08-22
1213   1931-05-11
1214   1987-03-13
Name: release_date, dtype: datetime64[ns]

## Anexo MONGODB

La primera aproximación para implementar la solución, fue hacerlo sobre MongoDB, sin embargo, la plataforma sólo soporta Redis en modo gratuito.

Se mantuvo la sección para usos futuros.

In [128]:
# Prueba 1) Ver si la bbdd responde
# Test de conexión

# String de conexión con la base de datos
client            = MongoClient('mongodb://localhost:27017')
db                = client['PI_srp']
movies_collection = db['work']

# Realizar la consulta
anio_test = 1996
query = {
    'release_year': anio_test                             # Consulta por películas estrenadas el año XXXX
        }
movies_count = movies_collection.count_documents(query)

# Imprimir el resultado
print(f"La cantidad de películas estrenadas el año {anio_test} fue de: {movies_count}")

La cantidad de películas estrenadas el año 1996 fue de: 633


In [129]:
# Prueba 2) Veces que se repite un Director

# String de conexión con la base de datos
client            = MongoClient('mongodb://localhost:27017')
db                = client['PI_srp']
collection        = db['work']

# Realizar la consulta
dire_test = 'Frank Capra'
query = {
    'director': dire_test                          
}
consulta = collection.count_documents(query)

# Retorna resultado
print(f"Menciones del director {dire_test}: {consulta}")

Menciones del director Frank Capra: 23


### Función que retorna: "Pelis X mes" en MongoDB

In [130]:
from pymongo import MongoClient

def pelis_por_mes(mes):
    # Mapear el nombre del mes a su valor numérico
    mapeo_meses = {
        'enero'     : 1,
        'febrero'   : 2,
        'marzo'     : 3,
        'abril'     : 4,
        'mayo'      : 5,
        'junio'     : 6,
        'julio'     : 7,
        'agosto'    : 8,
        'septiembre': 9,
        'octubre'   : 10,
        'noviembre' : 11,
        'diciembre' : 12
    }

    # Convertir el nombre del mes a minúsculas y obtener su valor numérico
    mes        = mes.lower()
    num_mes    = mapeo_meses.get(mes)

    if num_mes is None:
        raise ValueError('Nombre de mes inválido.')

    # String de conexión con la base de datos
    client     = MongoClient('mongodb://localhost:27017')
    db         = client['PI_srp']
    collection = db['movies']

    # Realizar la consulta
    query = {
        'month_release': num_mes
            }
    suma_pelis = collection.count_documents(query)

    # Retornar el resultado
    return print(f"La cantidad de películas estrenadas en el mes de {mes} es: {suma_pelis}")


In [131]:
pelis_por_mes('diciembre')

La cantidad de películas estrenadas en el mes de diciembre es: 3786


### Función que retorna: "Pelis X día" en MongoDB

In [132]:
# Función Pelis x día

def cantidad_filmaciones_dia(dia):
    """
    Recibe : Una cadena de texto con el nombre de un día.
    
    Retorna: La suma de películas estrenadas en ese mismo día.
    
    Ejemplo: "3612 películas fueron estrenadas un lunes".
    """
    # Mapea los días de la semana en español a su valor numérico
    dia_mapping = {
        'lunes'    : 1,
        'martes'   : 2,
        'miércoles': 3,
        'jueves'   : 4,
        'viernes'  : 5,
        'sábado'   : 6,
        'domingo'  : 7
    }

    # Convertir el día a minúsculas y obtener su valor numérico
    dia = dia.lower()

    if dia in dia_mapping:
        dia_numero = dia_mapping[dia]
    else:
        raise ValueError('Nombre de día inválido.')

    # Establecer la conexión con la base de datos
    client     = MongoClient('mongodb://localhost:27017')
    db         = client['PI_srp']
    collection = db['movies']

    # Realizar la consulta
    query = {
        'release_date': {
            '$exists': True
        },
        '$expr': {
            '$eq': [
                {'$dayOfWeek': '$release_date'},
                dia_numero
            ]
        }
    }
    movies_count = collection.count_documents(query)

    # Retornar el resultado
    return movies_count


In [133]:
# Test 
dia = 'lunes'
count = cantidad_filmaciones_dia(dia)
print(f"{count} películas fueron estrenadas un {dia}.")

3612 películas fueron estrenadas un lunes.


### Función que retorna: "Score x Peli" en MongoDB

In [134]:
# Función Score por título
# ************************

def score_titulo(nom_pel):
    # Convertir la cadena a tipo título
    nom_pel = nom_pel.title()

    # Establecer la conexión con la base de datos
    client            = MongoClient('mongodb://localhost:27017')
    db                = client['PI_srp']
    movies_collection = db['movies']

    # Realizar la consulta
    query = {
        'title': nom_pel
    }
    projection = {
        '_id'          : 0,
        'title'        : 1,
        'year_realease': 1,
        'popularity'   : 1
    }
    
    # Verificar si se encontró la película
    if movie := movies_collection.find_one(query, projection):
        titulo       = movie['title']
        anio_estreno = movie['year_realease']
        score        = movie['popularity']
        print(f"La película {titulo} fue estrenada en el año {anio_estreno}, con un score/popularidad de {score}")
    else:
        print(f"No se encontró la película {nom_pel}")

In [135]:
# Test
score_titulo('Star Wars')


La película Star Wars fue estrenada en el año 1977, con un score/popularidad de 42.149697


### Función que retorna: "Votos x Peli" en MongoDB

In [136]:
# Función votos por título
# ************************
def votos_titulo(titulo):
    # Convertir el título a minúsculas
    titulo = titulo.title()

    # String de conexión 
    client            = MongoClient('mongodb://localhost:27017')
    db                = client['PI_srp']
    movies_collection = db['movies']

    # Estructura de la consulta
    query = {
        'title': titulo
    }
    projection = {
        '_id'          : 0,
        'title'        : 1,
        'year_realease': 1,
        'vote_count'   : 1,
        'vote_average' : 1
    }
    movie = movies_collection.find_one(query, projection)

    # Verificar si se encontró la película
    if movie:
        titulo                       = movie['title']
        anio_estreno                 = movie['year_realease']
        cantidad_de_valoraciones     = movie['vote_count']
        promedio_de_las_valoraciones = movie['vote_average']

        if cantidad_de_valoraciones < 2000:
            print("Puntaje insuficiente!!")
        else:
            print(f"La película {titulo} fue estrenada en el año {anio_estreno}, cuenta con {cantidad_de_valoraciones} valoraciones, con un promedio de {promedio_de_las_valoraciones}")
    else:
        print(f"No se encontró la película {titulo}")
        
## Test 1

votos_titulo('Toy Story')

La película Toy Story fue estrenada en el año 1995, cuenta con 5415 valoraciones, con un promedio de 7.7


In [137]:
# Test 2

votos_titulo('Die Hard')

La película Die Hard fue estrenada en el año 1988, cuenta con 4005 valoraciones, con un promedio de 7.5


### Función que retorna: "Votos x Peli" V2 en MongoDB

In [138]:
# Función votos por película V2
# *****************************

def votos_titulo2(titulo):
    # Convertir el título a minúsculas
    titulo = titulo.title()

    # Establecer la conexión con la base de datos
    client            = MongoClient('mongodb://localhost:27017')
    db                = client['PI_srp']
    movies_collection = db['myc']

    # Realizar la consulta
    query = {
        'title': titulo
            }
    projection = {
        '_id'          : 0,
        'title'        : 1,
        'release_year' : 1,
        'vote_count'   : 1,
        'vote_average' : 1
    }
    movie = movies_collection.find_one(query, projection)

    titulo                       = movie['title']
    anio_estreno                 = movie['release_year']

    return print(f"La película {titulo} fue estrenada en el año {anio_estreno}")    

# Test 1

votos_titulo2('Toy Story')

La película Toy Story fue estrenada en el año 1995


In [139]:
votos_titulo2('Casablanca')


La película Casablanca fue estrenada en el año 1942


___
## Enriquecimiento de BBDD

### Función que agrega directores, elenco y otros datos, desde fuentes complementarias

Función que agrega las columnas: directores, elenco, generos, pais, productora, critica_cine al df de trabajo.
 
Recibe el df de trabajo y el df_imdb (base de datos de pelis), hace un cruce usando el código imdb y agrega información creando las columnas respectivas.

Se usa el código de pelicula 'ttXXXXXXX' para comparar los registros de ambos df y se copia el contenido de las celdas en el df de trabajo.

Motivo: La estrategia inicial fue trabajar con MongoDB, donde se implementaron las 4 primeras funciones, haciendo consultas a la colección 'movies' y 'credits'. La principal causa para cambiar es que, Mongo no es gratis en la plataforma de despliegue y no daba el tiempo para partir de cero con Redis.  

Este truco, permite además, resolver el problema de las dos últimas consultas que requerían hacer un pipeline que después de varios intentos no funcionaba y quitaba tiempo.

Finalmente, se optó por guardar los datos en formato excel que es, visiblemente, más eficiente para reducir el tamaño del archivo.

### Carga de datos externos

In [26]:
# Carga de datos externos

df_imdb = pd.read_csv('D:\BootcampHDS\M07_ProyInd\PI_DSPT01_0623\Ejemplos_hallados\IMDB-Business-Insights-SQLAndPython-main\IMDb movies.csv')

C:\Users\Estudio\AppData\Local\Temp\ipykernel_13724\1028361316.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb = pd.read_csv('D:\BootcampHDS\M07_ProyInd\PI_DSPT01_0623\Ejemplos_hallados\IMDB-Business-Insights-SQLAndPython-main\IMDb movies.csv')


In [27]:
# Muestreo IMDB
por_nombre = df_imdb[df_imdb['imdb_title_id'] == 'tt0114709' ]
por_nombre

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
29371,tt0114709,Toy Story - Il mondo dei giocattoli,Toy Story,1995,1996-03-22,"Animation, Adventure, Comedy",81,USA,English,John Lasseter,"John Lasseter, Pete Docter",Pixar Animation Studios,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",A cowboy doll is profoundly threatened and jea...,8.3,864461,$ 30000000,$ 222498679,$ 404265438,95.0,617.0,160.0


In [28]:
# Muestreo Data Original
df_movies_or.head(1)

,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return
0,"{""id"": 10194, ""name"": ""Toy Story Collection"", ...",30000000.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""...",862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-10-30,373554033.0,81.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Sin dato,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801


### Función que enriquece BBDD original

In [29]:
# Función que enriquece la BBDD original
# **************************************
# Utilizando otra base sobre IMDB, se complementan
# datos sobre actores, directores y géneros.
# La función recibe un dataset externo (input)
# y un dataset (output), set de datos original 
# modificado, con la columnas agregadas.
# toma el id del imdb y compara con el df externo
# Si ambos datos coinciden, copia los datos y
# los agrega a las columnas creadas.
# Los archivos json, generaban una serie de problemas
# y esta fue la forma de resolverlo, haciendo cruce
# de datos.

def f_enriquece_bbdd(df_input, df_out):
    """
    Función que enriquece el df procesado.
    
    Recibe  : Un df externo (input) y el df
    de trabajo (output).
    
    Retorna : El df output, con 3 nuevas columnas
    y los datos necesarios para hacer otras consultas
    
    """
    # Verificar la cantidad de filas al inicio
    original_rows = df_out.shape[0]
    
    # Agregar columnas 'director', 'elenco' y 'generos'al dataframe 'df_out'
    df_out['director']     = None
    df_out['elenco']       = None
    df_out['generos']      = None
    df_out['pais']         = None
    df_out['productora']   = None
    df_out['critica_cine'] = None

    # Recorrer el dataframe 'df_out' y combinar los datos de 'df_imdb'
    for index, row in df_out.iterrows():
        movie_id     = row['imdb_id']
        misma_fila = df_input[df_input['imdb_title_id'] == movie_id]
        if not misma_fila.empty:
            dires      = misma_fila['director'].values[0]
            actores    = misma_fila['actors'].values[0]
            generos    = misma_fila['genre'].values[0]
            pais       = misma_fila['country'].values[0]
            productora = misma_fila['production_company'].values[0]
            criticos   = misma_fila['reviews_from_critics'].values[0]
            df_out.at[index, 'director']     = dires
            df_out.at[index, 'elenco']       = actores
            df_out.at[index, 'generos']      = generos
            df_out.at[index, 'pais']         = pais
            df_out.at[index, 'productora']   = productora
            df_out.at[index, 'critica_cine'] = criticos

    # Verificar la cantidad de filas después del proceso
    final_rows = len(df_out)

    # Auditoría de cambios
    print(f"Filas originales: {original_rows}")
    print(f"Filas finales   : {final_rows}")

    return df_out

In [30]:
f_enriquece_bbdd(df_imdb, df_movies_or)

Filas originales: 45355
Filas finales   : 45355


,belongs_to_collection,budget,genres,id,imdb_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return,director,elenco,generos,pais,productora,critica_cine
0,"{""id"": 10194, ""name"": ""Toy Story Collection"", ...",30000000.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""...",862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-10-30,373554033.0,81.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Sin dato,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Animation, Adventure, Comedy",USA,Pixar Animation Studios,160.0
1,"{""id"": 999999, ""name"": ""Sin dato""}",65000000.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.015539,"[{""name"": ""TriStar Pictures"", ""id"": 559}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-12-15,262797249.0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,12.0,15.0,5.0,4.043035,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","Adventure, Comedy, Family",USA,TriStar Pictures,73.0
2,"{""id"": 119050, ""name"": ""Grumpy Old Men Collect...",0.0,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-12-22,0.0,101.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,12.0,22.0,5.0,0.000000,Howard Deutch,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...","Comedy, Romance",USA,Lancaster Gate,29.0
3,"{""id"": 999999, ""name"": ""Sin dato""}",16000000.0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-12-22,81452156.0,127.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,12.0,22.0,5.0,5.090760,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...","Comedy, Drama, Romance",USA,Twentieth Century Fox,29.0
4,"{""id"": 96871, ""name"": ""Father of the Bride Col...",0.0,"[{""id"": 35, ""name"": ""Comedy""}]",11862,tt0113041,en,Just when George Banks has recovered from his ...,8.387519,"[{""name"": ""Sandollar Productions"", ""id"": 5842}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-02-10,76578911.0,106.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Just When His World Is Back To Normal... He""s ...",Father of the Bride Part II,5.7,173.0,1995.0,2.0,10.0,5.0,inf,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...","Comedy, Family, Romance",USA,Sandollar Productions,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45350,"{""id"": 999999, ""name"": ""Sin dato""}",0.0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...",30840,tt0102797,en,"Yet another version of the classic epic, with ...",5.683753,"[{""name"": ""Westdeutscher Rundfunk (WDR)"", ""id""...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",1991-05-13,0.0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Sin dato,Robin Hood,5.7,26.0,1991.0,5.0,13.0,1.0,0.00

In [39]:
# Volvemos a chequear los cambios y vemos si hay valores nulos

df_movies_or.isnull().sum()

budget               0
id                   0
imdb_id              0
original_language    0
overview             0
popularity           0
release_date         0
revenue              0
runtime              0
status               0
tagline              0
title                0
vote_average         0
vote_count           0
release_year         0
release_month        0
release_day          0
release_weekday      0
return               0
director             0
elenco               0
generos              0
pais                 0
productora           0
critica_cine         0
dtype: int64

### Corrige valores nulos después de enriquecer BBDD - Parte 2

In [40]:
# Función que corrige valores nulos
# *********************************
# Reusamos la lógica de la función que corrige el df 
# Se ajusta el motor de reglas que recopila los 
# cambios en las nuevas columnas, verifica tipos, 
# agrega texto, valor 0 o datos json.
#
# Recibe  : El df
#
# Retorna : El df procesado

def corregir_valores_nulos2(df):
    # Regla 1: Chequear y reemplazar valores nulos en "director"
    if 'director' in df.columns:
        df['director'] = df['director'].fillna("Sin dato")
    
    # Regla 2: Reemplazar valores nulos en "elenco" con "Sin dato"
    if 'elenco' in df.columns:
        df['elenco'] = df['elenco'].fillna("Sin dato")
    
    # Regla 3: Reemplazar valores nulos en "generos" con "Sin dato"
    if 'generos' in df.columns:
        df['generos'] = df['generos'].fillna("Sin dato")
    
    # Regla 4: Reemplazar valores nulos en "pais" con "Sin dato"
    if 'pais' in df.columns:
        df['pais'] = df['pais'].fillna("Sin dato")
    
    # Regla 5: Reemplazar valores nulos en "productora" con "Sin dato"
    if 'productora' in df.columns:
        df['productora'] = df['productora'].fillna("Sin dato")
    
    # Regla 6: Reemplazar valores nulos en "critica_cine" con 0
    if 'critica_cine' in df.columns:
        df['critica_cine'] = df['critica_cine'].fillna(0)
    
    # Regla 7: Generar reporte de auditoría
    cambios = df.isna().sum()
    reporte_cambios = pd.DataFrame(cambios, columns=['Valores Nulos'])
    reporte_cambios['Valores Actualizados'] = len(df) - cambios
    reporte_cambios.columns = ['Valores Nulos', 'Valores Actualizados']
    
    return df, reporte_cambios

In [41]:
df_movies_or, reporte_cambios  = corregir_valores_nulos2(df_movies_or)

In [42]:
# Auditoría de cambios
reporte_cambios 

,Valores Nulos,Valores Actualizados
budget,0,45355
id,0,45355
imdb_id,0,45355
original_language,0,45355
overview,0,45355
popularity,0,45355
release_date,0,45355
revenue,0,45355
runtime,0,45355
status,0,45355


### Elimina columnas que no se van a utilizar

El enriquecimiento de la base de datos nos ayuda y resuelve, el problema de disminuir el peso de la base de datos al prescindir del set de datos 'credits.csv'.

In [ ]:
# Eliminación de columnas.

df_movies_or = df_movies_or.drop(['spoken_languages', 'production_countries', 'production_companies', 'belongs_to_collection', 'genres'], axis=1)

In [36]:
df_movies_or.head(1)

,budget,id,imdb_id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return,director,elenco,generos,pais,productora,critica_cine
0,30000000.0,862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Sin dato,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Animation, Adventure, Comedy",USA,Pixar Animation Studios,160.0


In [44]:
df_imdb.head(1)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0


## Desarrollo funciones en PANDAS

### Función pelis x día adaptada a Pandas

In [47]:
# FUNCION PELIS X DIA ADAPTADA A PANDAS

def f_cantidad_filmaciones_dia(v_Dia):
    """
    Recibe  : Una cadena de texto con el nombre un díade la semana.
    
    Excepciones : Si no está en el dic, arroja error.
    
    Retorna: La suma de películas estrenadas el día ingresado.
    """
    # Mapea los días de la semana en español a su valor numérico
    dic_map_dias = {
            'lunes'    : 1,
            'martes'   : 2,
            'miércoles': 3,
            'jueves'   : 4,
            'viernes'  : 5,
            'sábado'   : 6,
            'domingo'  : 7
                    }

    # Convierte una cadena de texto que representa el día a minúsculas
    # y obtiene su valor numérico
    v_Dia = v_Dia.lower()  # Convierte la cadena a minúsculas

    if v_Dia in dic_map_dias:  # Verifica que la cadena v_Dia esté en el dic
        v_dia_num = dic_map_dias[v_Dia]
    else:
        raise ValueError('Nombre de día inválido.')

    # Filtra el dataframe por el día de la semana
    df_consulta = df_work[df_work['release_weekday'] == v_dia_num]

    v_suma_pelis = len(df_consulta)  # Cuenta la cantidad de registros en el dataframe filtrado


    print(f"{v_suma_pelis} películas fueron estrenadas un {v_Dia}.")
    return 

# Test
f_cantidad_filmaciones_dia('viernes')


13905 películas fueron estrenadas un viernes.


In [ ]:
def contar_pelis_por_dia(v_Dia):
    # Crear diccionario semana
    dic_semana = {
        'lunes'    : 1,
        'martes'   : 2,
        'miércoles': 3,
        'jueves'   : 4,
        'viernes'  : 5,
        'sábado'   : 6,
        'domingo'  : 7
            }

    # Validar 'v_Dia' y convertir cadena a minúscula
    v_Dia = v_Dia.lower()

    while v_Dia not in dic_semana:
        v_Dia = input("Ingrese un día de la semana válido: ").lower()

    # Guardar valor temporal
    v_tmp_num = dic_semana[v_Dia]

    # Contar películas por día de la semana
    v_suma_pelis = df_work[df_work['release_weekday'] == v_tmp_num]['release_weekday'].count()

    # Imprimir resultado
    #print(f"{v_suma_pelis} películas fueron estrenadas un {v_Dia}.")

    # Retornar cantidad de películas
    return print(f"{v_suma_pelis} películas fueron estrenadas un dia {v_Dia}.")#v_suma_pelis

# Test

contar_pelis_por_dia('viernes')

13905 películas fueron estrenadas un dia viernes.


### Función pelis x mes version Pandas

In [48]:
def f_cantidad_filmaciones_mes2(v_Mes):
    """
    Recibe: una cadena de texto con el nombre del mes
    en idioma español.
    
    No valida el idioma, sólo que pertenezca al 
    diccionario.
    
    Si la cadena es inválida, solicitará
    otra cadena válida.
    
    Retorna: La cantidad de películas estrenadas en el mes 'X'
    """
    
    # Crear diccionario con los nombres de los meses y su valor numérico
    # Mapea la cadena ingresada y si pertenece, la convierte a su valor numérico
    dic_map_meses = {
        'enero'     : 1,
        'febrero'   : 2,
        'marzo'     : 3,
        'abril'     : 4,
        'mayo'      : 5,
        'junio'     : 6,
        'julio'     : 7,
        'agosto'    : 8,
        'septiembre': 9,
        'octubre'   : 10,
        'noviembre' : 11,
        'diciembre' : 12
    }

    # Para controlar las cadenas ingresadas, primero se convierten a minúsculas
    # luego, se obtiene su valor numérico
    v_Mes    = v_Mes.lower()
    v_numMes = dic_map_meses.get(v_Mes)

    # Valida el ingreso de cadenas inválidas 
    if v_numMes is None:                               
        raise ValueError('Mes ingresado es inválido.')

    # Realizar la consulta en el DataFrame
    query       = df_work['release_month'] == v_numMes
    v_num_pelis = len(df_work[query])

    # Imprime el resultado de la consulta
    return print(f"La cantidad de películas estrenadas en el mes de {v_Mes} es: {v_num_pelis}")

f_cantidad_filmaciones_mes2('mayo')


La cantidad de películas estrenadas en el mes de mayo es: 3337


### Función score x peli vesión Pandas

In [49]:
import pandas as pd

def score_titulo(v_nom_pel):
    """
    Recibe: una cadena de texto con el nombre de
    una película en su idoma original.
    
    No valida el idioma, sólo que pertenezca al 
    la serie 'título'.
    
    Si la cadena es inválida, solicitará
    otra cadena válida.
    
    Retorna: Nombre de la pelicula, año de estreno, popularidad.
    
    Ejemplo : La película Dracula fue estrenada en el año 1992, con un score/popularidad de 16.777675
    """
    
    
    # Convertir la cadena a tipo título
    v_nom_pel = v_nom_pel.title()

    # Realizar la consulta en el DataFrame
    query = df_work['title'] == v_nom_pel
    movie = df_work.loc[query, ['title', 'release_year', 'popularity']].head(1)

    # Verificar si se encontró la película
    if not movie.empty:
        titulo = movie['title'].iloc[0]
        anio_estreno = movie['release_year'].iloc[0]
        score = movie['popularity'].iloc[0]
        print(f"La película {titulo} fue estrenada en el año {int(anio_estreno)}, con un score/popularidad de {score}")
    else:
        print(f"No se encontró la película {v_nom_pel}")

# Test

score_titulo('Dracula')

La película Dracula fue estrenada en el año 1992, con un score/popularidad de 16.777675


### Función get_actor(nombre_actor)

In [50]:
# Para no cambiar toda la lógica que había trabajado 
# con un df llamado df_work, le asignamos el mismo
# nombre al df activo.

df_work = df_movies_or

In [51]:
# Función: 'get_actor', retorna # de pelis, revenue total y promedio.
# *******************************************************************
# Función de consulta que retorna el número de películas en 
# las que un actor ha participado y su éxito, medido como la
# suma de ingresos (revenue) de sus películas, siempre que 
# exista esa información, y el promedio de ingresos, expresado
# como el total de ingresos, dividido por la cantidad de películas 
# en las que ha participado.
# 
# Recibe: Una cadena de texto, nombre del actor. No valida 
# variantes del nombre, ni su grafía, es decir, si 
# encuentra una cadena igual, retornará los valores de la columna
# 'revenue' y realizará el cálculo del promedio.
# 
# Retorna: Imprime un mensaje que dice 'El actor X ha participado 
# de Y cantidad de filmaciones, con un un promedio de Z por filmación', 
# donde X es el nombre del actor, Y el acumulado de 'revenue', 
# y Z, el promedio de 'revenue' por película.
#
# La función busca la cadena de texto dentro de una lista de nombres, 
# en el campo 'elenco', para cuál "divide la lista" (split).

# Ejemplo get_actor('Tom Hanks') 
# Retorna: 'El actor Tom Hanks, ha participado en 35 filmaciones, el 
# mismo ha conseguido un retorno acumulado de 7556298727, con un 
# promedio de 215894249.34285715 por filmación'.

def get_actor(v_nom_actor):
    """
    Función que busca un actor y retorna el número de películas en que
    ha participado, el acumulado de ingresos y su promedio.
    
    Recibe: Una cadena de texto, que es el nombre de un actor.
    
    Retorna: Nombre del actor, número de películas en las que participó,
    el acumulado de ingresos 'revenue' y el promedio ('revenue'/número de películas)
    
    Imprime: 'El actor X ha participado de Y cantidad de filmaciones, con 
    un un promedio de Z por filmación'
    """
    # Inicializa contadores y acumulador
    v_contador    = 0
    v_sum_revenue = 0
    
    # Covierte la cadena a tipo título que es el formato del dato 
    v_nom_actor = v_nom_actor.title()
    for v_elenco in df_work['elenco']:
        try:
            if v_nom_actor in v_elenco.split(','):
                v_contador    += 1
                v_revenue      = df_work.loc[df_work['elenco'] == v_elenco, 'revenue'].values[0]
                v_sum_revenue += v_revenue
        except AttributeError:
            # Manejar la excepción cuando el valor es numérico (float)
            continue
    
    # Formateo de los valores
    v_contador_format    = '{:,}'.format(v_contador)
    v_sum_revenue_format = '${:,.2f}'.format(v_sum_revenue)
    v_prom_revenue       = '${:,.2f}'.format(v_sum_revenue / v_contador)
    
    # Respuesta formateada
    print(f"Actor/Actriz {v_nom_actor}")
    print(f"Número de producciones en que ha participado : {v_contador_format}")
    print(f"Acumulado de revenue                         : {v_sum_revenue_format}")
    print(f"Promedio generado por película               : {v_prom_revenue}")


get_actor("Tom Hanks")

Actor/Actriz Tom Hanks
Número de producciones en que ha participado : 35
Acumulado de revenue                         : $7,556,298,727.00
Promedio generado por película               : $215,894,249.34


In [52]:
df_movies_or.head(1)


,budget,id,imdb_id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return,director,elenco,generos,pais,productora,critica_cine
0,30000000.0,862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Sin dato,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Animation, Adventure, Comedy",USA,Pixar Animation Studios,160.0


### Función get_director(nombre_director)

Función que recibe una cadena de texto con el nombre de un director de cine, retorna el nombre del director, la cantidad de películas que ha dirigido, y una lista con los títulos de sus películas, año de estreno, presupuesto o costo, ingresos y utilidad (revenue - budget).

In [53]:
# Función get_director()
# **********************
# Consulta el nombre de un director y retorna información
# sobre sus películas.

def get_director(nom_dire):
    """
    Función de consulta.
    
    Recibe  : Cadena de texto que el nombre de un director de cine.
    
    No valida grafía, si la cadena coincide, realiza la búsqueda.
    
    Retorna : Nombre del director, las películas dirigidas, luego
    genera una lista con el nombre de cada película, año de lanzamiento,
    presupuesto, ingresos y utilidad.
    """
    num_title = 0
    peliculas = []

    for i in range(len(df_work)):
        director = df_work.loc[i, 'director']

        if isinstance(director, str):
            if nom_dire in director.split(','):
                num_title   += 1
                title        = df_work.loc[i, 'title']
                release_year = df_work.loc[i, 'release_year']
                budget       = df_work.loc[i, 'budget']
                revenue      = df_work.loc[i, 'revenue']
                utilidad     = revenue - budget

                peliculas.append((title, release_year, budget, revenue, utilidad))

    # Imprimir resultados formateados
    print(f"El Director {nom_dire} ha dirigido {num_title} producciones")
    print("La lista de películas, con sus años de lanzamiento, costo, ingresos y utilidad, es la siguiente:")
    print("***  Nombre de la película  *** año de lanzamiento *** Costo (USD) *** Ingresos (USD) *** Utilidad (USD)")
    
    for peli in peliculas:
        title, release_year, budget, revenue, utilidad = peli
        print(f"{title} *** {int(release_year)} *** ${budget:,.2f} *** ${revenue:,.2f} *** ${utilidad:,.2f}")


In [54]:
get_director('Joe Johnston')

El Director Joe Johnston ha dirigido 9 producciones
La lista de películas, con sus años de lanzamiento, costo, ingresos y utilidad, es la siguiente:
***  Nombre de la película  *** año de lanzamiento *** Costo (USD) *** Ingresos (USD) *** Utilidad (USD)
Jumanji *** 1995 *** $65,000,000.00 *** $262,797,249.00 *** $197,797,249.00
Honey, I Shrunk the Kids *** 1989 *** $32,000,000.00 *** $222,724,172.00 *** $190,724,172.00
The Rocketeer *** 1991 *** $42,000,000.00 *** $62,000,000.00 *** $20,000,000.00
October Sky *** 1999 *** $0.00 *** $0.00 *** $0.00
Jurassic Park III *** 2001 *** $93,000,000.00 *** $368,780,809.00 *** $275,780,809.00
Hidalgo *** 2004 *** $100,000,000.00 *** $108,103,450.00 *** $8,103,450.00
The Wolfman *** 2010 *** $150,000,000.00 *** $0.00 *** $-150,000,000.00
Captain America: The First Avenger *** 2011 *** $140,000,000.00 *** $370,569,774.00 *** $230,569,774.00
Not Safe for Work *** 2014 *** $0.00 *** $0.00 *** $0.00


Insight: La salida sin formato tabular...es horrible!!, intentaremos con la librería 'tabulate'

### Prueba de concepto uso de librería TABULATE

In [55]:
# Ejemplo librería 'tabulate'
# ***************************
# Se pueden ajustar múltiples parámetros

# Datos de ejemplo
datos_ej =  [
    {"Nombre": "Juan Perez", "Edad"   : 28, "Ciudad": "Nueva York"},
    {"Nombre": "John Wayne", "Edad"   : 35, "Ciudad": "Chicago"},
    {"Nombre": "Ben Kingsley", "Edad" : 42, "Ciudad": "Los Ángeles"}
            ]

# Aplica las transformaciones
table = tabulate(datos_ej, headers="keys", tablefmt="grid")

# Resultado
print(table)

+--------------+--------+-------------+
| Nombre       |   Edad | Ciudad      |
+==============+========+=============+
| Juan Perez   |     28 | Nueva York  |
+--------------+--------+-------------+
| John Wayne   |     35 | Chicago     |
+--------------+--------+-------------+
| Ben Kingsley |     42 | Los Ángeles |
+--------------+--------+-------------+


In [56]:
# Función get_director() mejorada, versión TABULATE
# *************************************************
# Retorna: # de pelis, revenue total y promedio.
# 
# Función de consulta que retorna el número de películas que un 
# director ha dirigido y su éxito, medido como la utilidad por
# cada película, que es igual a los ingresos menos el presupuesto.
# 
# Recibe: Una cadena de texto, nombre del director. No valida 
# variantes del nombre, ni su grafía, es decir, si encuentra 
# una cadena igual, realiza la consulta

# Retorna: El nombre del director, cantidad de películas que ha 
# dirigido, y una lista con los nombres de cada película, su año 
# de estreno, los ingresos, el presupuesto, y la utilidad, que es 
# igual a la diferecia entre los ingresos y los costos.
# 
# Imprime un mensaje que dice:
# 
# El Director {v_nom_dire} ha dirigido {num_title} producciones.
# La lista de películas, con sus respectivas fechas de lanzamiento, costo, ingresos y utilidad, es la siguiente:
#
#El Director Joe Johnston ha dirigido 9 producciones.
#La lista de películas, con sus respectivas fechas de lanzamiento, costo, ingresos y utilidad, es la siguiente:
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Nombre Película                    | Fecha Lanzamiento   | Costo (USD)   | Ingresos (USD)   | Utilidad (USD)   |
#+====================================+=====================+===============+==================+==================+
#| Jumanji                            | 1995-12-15          | $65000000.00  | $262797249.00    | $197797249.00    |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Honey, I Shrunk the Kids           | 1989-06-22          | $32000000.00  | $222724172.00    | $190724172.00    |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| The Rocketeer                      | 1991-06-21          | $42000000.00  | $62000000.00     | $20000000.00     |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| October Sky                        | 1999-02-19          | $0.00         | $0.00            | $0.00            |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Jurassic Park III                  | 2001-07-18          | $93000000.00  | $368780809.00    | $275780809.00    |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Hidalgo                            | 2004-03-05          | $100000000.00 | $108103450.00    | $8103450.00      |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| The Wolfman                        | 2010-02-11          | $150000000.00 | $0.00            | $-150000000.00   |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Captain America: The First Avenger | 2011-07-22          | $140000000.00 | $370569774.00    | $230569774.00    |
#+------------------------------------+---------------------+---------------+------------------+------------------+
#| Not Safe for Work                  | 2014-05-09          | $0.00         | $0.00            | $0.00            |
#+------------------------------------+---------------------+---------------+------------------+------------------+

def f_get_director(v_nom_dire):
    """
    Función de consulta.
    
    Recibe  : Cadena de texto que el nombre de un director de cine.
    
    No valida grafía y, si la cadena coincide, realiza la búsqueda.
    
    Retorna : Nombre del director, las películas dirigidas, luego
    genera una lista con el nombre de cada película, año de lanzamiento,
    presupuesto, ingresos y utilidad.
    """
    num_title = 0
    peliculas = []

    for i in range(len(df_work)):
        director = df_work.loc[i, 'director']

        if isinstance(director, str):
            if v_nom_dire in director.split(','):
                num_title += 1
                title = df_work.loc[i, 'title']
                release_date = df_work.loc[i, 'release_date']
                budget = df_work.loc[i, 'budget']
                revenue = df_work.loc[i, 'revenue']
                utilidad = revenue - budget

                peliculas.append((title, release_date, budget, revenue, utilidad))

    # Imprimir resultados formateados
    print(f"El Director {v_nom_dire} ha dirigido {num_title} producciones.")
    print("La lista de películas, con sus respectivas fechas de lanzamiento, costo, ingresos y utilidad, es la siguiente:")
    
    # Estructura tabla
    lst_nom_cols = ['Nombre Película', 'Fecha Lanzamiento', 'Costo (USD)', 'Ingresos (USD)', 'Utilidad (USD)']
    lst_formateada = []
    for fila in peliculas:
        fila_formateada = [
        fila[0],
        datetime.strptime(str(fila[1]), "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d"),
        "${:.2f}".format(fila[2]),
        "${:.2f}".format(fila[3]),
        "${:.2f}".format(fila[4])
                        ]
        lst_formateada.append(fila_formateada)

    peliculas = tabulate(lst_formateada, headers = lst_nom_cols, tablefmt = "grid")   
    print(peliculas)
    return 

In [57]:
f_get_director('John Woo')

El Director John Woo ha dirigido 18 producciones.
La lista de películas, con sus respectivas fechas de lanzamiento, costo, ingresos y utilidad, es la siguiente:
+--------------------------+---------------------+---------------+------------------+------------------+
| Nombre Película          | Fecha Lanzamiento   | Costo (USD)   | Ingresos (USD)   | Utilidad (USD)   |
+==========================+=====================+===============+==================+==================+
| Broken Arrow             | 1996-02-09          | $50000000.00  | $150270147.00    | $100270147.00    |
+--------------------------+---------------------+---------------+------------------+------------------+
| Hard Target              | 1993-08-20          | $18000000.00  | $74189677.00     | $56189677.00     |
+--------------------------+---------------------+---------------+------------------+------------------+
| The Killer               | 1989-07-06          | $0.00         | $0.00            | $0.00            |

## Función get_director()

In [58]:
f_get_director('Frank Capra')

El Director Frank Capra ha dirigido 25 producciones.
La lista de películas, con sus respectivas fechas de lanzamiento, costo, ingresos y utilidad, es la siguiente:
+-------------------------------+---------------------+---------------+------------------+------------------+
| Nombre Película               | Fecha Lanzamiento   | Costo (USD)   | Ingresos (USD)   | Utilidad (USD)   |
+===============================+=====================+===============+==================+==================+
| It Happened One Night         | 1934-02-22          | $325000.00    | $4500000.00      | $4175000.00      |
+-------------------------------+---------------------+---------------+------------------+------------------+
| Lost Horizon                  | 1937-09-01          | $4000000.00   | $0.00            | $-4000000.00     |
+-------------------------------+---------------------+---------------+------------------+------------------+
| It"s a Wonderful Life         | 1946-12-20          | $3180000.0

### Función que abre PDF con el README del Proyecto.

Pensando en una interfaz tipo tkinter, se agregaron algunas funciones complementarias

In [59]:
# Función abrir PDF con README v1
# *******************************
# Pensando en una interfaz basada en Tkinter
# Permite abrir el readme en formato pdf.

def open_readme():
    """
    Función que abre un archivo README.pdf en el navegador 
    Google Chrome o en la aplicación para archivos PDF por defecto
    """
    ruta = '/docs'                           # La ruta de la carpeta del proyecto
    path = os.path.join(ruta, "README.pdf")  #
    operating_system = platform.system()     #

    if operating_system == "Windows":
        # Abrir en Google Chrome
        webbrowser.register('chrome', None, webbrowser.BackgroundBrowser("\\Program Files\\Google\\Chrome\\Application\\chrome.exe"))
        webbrowser.get('chrome').open(path)
    else:
        # Si el sistema operativo no es reconocido, abrir en el navegador por defecto
        webbrowser.open(path)



In [213]:
open_readme()

In [214]:
# Función abrir PDF con README v2
# *******************************
# Pensando en una interfaz basada en Tkinter
# Permite abrir el readme en formato pdf.
def open_pdf():
    """
    Función que abre un archivo README.pdf en el navegador 
    Google Chrome o en la aplicación para archivos PDF por defecto
    """
    
    path = os.path.join("rawdata/docs", "README.pdf") # Define el path 
    url  = f"https://docs.google.com/viewer?url=file://{os.path.abspath(path)}" # URL a ejecutar
    webbrowser.get('chrome').open(url) 

In [215]:
open_pdf()

___
## Guardar en distintos formatos

### Guardar en CSV

In [203]:
# A stagedata

df_work.to_csv(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\movies_300623.csv', index= False, encoding='utf-8')

In [ ]:
# A ML

df_work.to_csv(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\ML\\movies_300623.csv', index= False, encoding='utf-8')

### Guardar en EXCEL

In [ ]:
# A stagedata
df_movies_or.to_excel(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\dfwork300623.xlsx', index=False)


In [290]:
# A ML
df_movies_or.to_excel(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\ML\\dfwork300623.xlsx', index=False)


# HASTA AQUÍ CON EL ETL

___

## Pruebas FASTAPI

In [ ]:
# Primera aproximación
# ********************
# Guardar en un archivo para poder probar en dearrollo

import uvicorn

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

uvicorn nombre_archivo:app --reload

#### Resumen requerimientos para FASTAPI

Para disponibilizar los datos del dataframe df, usando el framework FastAPI, se requieren 6 funciones para los endpoints que se consumirán en la API, cada función debe tener un decorador (@app.get(‘/’)).

Función 1: 'cantidad_filmaciones_mes', Recibe una cadena de texto, que es un mes en idioma español (ejemplo: 'enero'). Retorna una consulta al dataframe 'df_movies', con la cantidad de películas que fueron estrenadas para el mes consultado, dentro de la totalidad del dataset.

Función 2: 'cantidad_filmaciones_dia': Recibe una cadena de texto, que es el nombre del día en idioma Español (ejemplo: 'lunes'). Retorna la cantidad de películas que fueron estrenadas para el día consultado en la totalidad del dataset.  Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

Función 3: 'score_titulo': Recibe una cadena de texto, que es el título de una película (ejemplo: 'Lo que el viento se llevó'). Retorna el título que se encuentra en la columna 'title', el año de estreno, que se encuentra en la columna 'year_release', y el score, que está en la columna 'popularity'.

- Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X. Si no encneuntra la cadena de texto, debe mostrar un mensaje que diga 'título no hallado, le sugiero estos:' y mostrar una lista que contenga, el título, el año de estreno y el score, de los títulos que mas se parezcan a la cadena de texto.

Función 4: 'votos_titulo': Recibe una cadena de texto que corresponde al título de una película, la que comparará y buscará en la columna 'title'. Al encontrar la coincidencia, deberá comprobar que tiene, al menos, 2000 puntos en la columna 'vote_count'. Si tiene menos de 2000 puntos, Retorna el título de la película, la cantidad de votos que tiene en la columna 'vote_count', y el valor promedio de las votaciones que se encuentran en la columna 'vote_average'. Si la película encontrada en la columna 'title', tiene menos de 2000 puntos en la columna 'vote_count', La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

- Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

Función 5: 'get_actor': Recibe una cadena de texto, que corresponde al nombre de un actor, que se encuentre dentro de un dataset llamado 'df_cast'. debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.

- Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

Función 6: 'get_director': Recibe una cadena de texto que corresponde al nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

Función 7: recomendacio(), ingresa una película y recomienda 5.

código para acceder a IMDB... para usos futuros

### Prueba de concepto conexión API 'themoviedb.org' para hacer actualizaciones dinámicas

In [ ]:
# Prueba 'themoviedb.org"
# ***********************
# Realizar una solicitud GET a la API de Themoviedb para obtener información sobre las películas
nombre  = "arnold"                             # Reemplazar con el valor deseado
api_key = "c7f7381bc44cd24b332ccc18f24fc126"   # Reemplazar con tu clave de API

url_api = f"https://api.themoviedb.org/3/search/person?api_key={api_key}&search_type=ngram&query={nombre}"

response = requests.get(url_api)
data     = response.json()
print (json.dumps(data, indent=4))

# Dar formato al JSON
#data = json.dumps(data, indent=4)

# Acceder a los resultados y recorrerlos
for movie in data['results']:
    # Acceder a las propiedades de cada película
    movie_id     = movie['id']
    title        = movie['title']
    poster_path  = movie['poster_path']
    
    # Imprimir las propiedades
    print(f'ID:,{movie_id}')
    print(f'Título original:, {title}')
    print(f'Ruta del póster:, {poster_path}')
    print("---")

___

## Zona de pruebas

In [81]:
import json

# Objeto JSON sin formato
json_data = {"name": "John", "age": 30, "city": "New York"}

# Dar formato al JSON
formatted_json = json.dumps(json_data, indent=4)

# Imprimir el JSON formateado
print(formatted_json)


{
    "name": "John",
    "age": 30,
    "city": "New York"
}


In [ ]:
db.movies.find(
    {$and :[
        {"cast" : "Charles Chaplin"},
        {"cast": "Edna Purviance"}
    ]},
    {"cast": 1, "_id": 0}
)

In [91]:
from pymongo import MongoClient

# Conexión a la base de datos
client     = MongoClient('mongodb://localhost:27017')
db         = client['PI_srp']
collection = db['movies']

# Definición de la consulta
query = {
    "$and": [
        {"title": "Jumamji"}
        #{"cast.name": "Edna Purviance"}
    ]
}
projection = {
    "title": 1,
    "_id": 0
}

# Ejecución de la consulta
results = collection.find(query, projection)

# Iteración sobre los resultados
for result in results:
    print(result)


In [110]:
df_credits.head(1)
import json

# Objeto JSON sin formato
json_data = {"name": "John", "age": 30, "city": "New York"}

# Dar formato al JSON
formatted_json = json.dumps(json_data, indent=4)

# Imprimir el JSON formateado
print(formatted_json)

{
    "name": "John",
    "age": 30,
    "city": "New York"
}


In [111]:
import json

# Definición del objeto JSON
data = {
    "name": "John Doe",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "New York",
        "country": "USA"
    },
    "hobbies": ["reading", "cooking", "traveling"],
    "languages": {
        "primary": "English",
        "secondary": ["Spanish", "French"]
    }
}

# Formatear la impresión del objeto JSON
formatted_json = json.dumps(data, indent=4)

# Imprimir el objeto JSON formateado
print(formatted_json)


{
    "name": "John Doe",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "New York",
        "country": "USA"
    },
    "hobbies": [
        "reading",
        "cooking",
        "traveling"
    ],
    "languages": {
        "primary": "English",
        "secondary": [
            "Spanish",
            "French"
        ]
    }
}


In [116]:
data = df_credits.cast.iloc[0]

In [117]:
print (data)

[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tN

In [120]:
import json

# Definición del objeto JSON
#data = {
#    "name": "John Doe",
#    "age": 30,
#    "address": {
#        "street": "123 Main St",
#        "city": "New York",
#        "country": "USA"
#    },
#    "hobbies": ["reading", "cooking", "traveling"],
#    "languages": {
#        "primary": "English",
#        "secondary": ["Spanish", "French"]
#    }
#}

# Formatear la impresión del objeto JSON
formatted_json = json.dumps(data, indent=4)

# Imprimir el objeto JSON formateado
ppjson(formatted_json)


"\"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH

In [ ]:
json_data = '[{"cast_id": 14, "character": "Woody (voice)", "credit_id": "52fe4284c3a36847f8024f95", "gender": 2, "id": 31, "name": "Tom Hanks", "order": 0, "profile_path": "/pQFoyx7rp09CJTAb932F2g8Nlho.jpg"}, {"cast_id": 15, "character": "Buzz Lightyear (voice)", "credit_id": "52fe4284c3a36847f8024f99", "gender": 2, "id": 12898, "name": "Tim Allen", "order": 1, "profile_path": "/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg"}, {"cast_id": 16, "character": "Mr. Potato Head (voice)", "credit_id": "52fe4284c3a36847f8024f9d", "gender": 2, "id": 7167, "name": "Don Rickles", "order": 2, "profile_path": "/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg"}, {"cast_id": 17, "character": "Slinky Dog (voice)", "credit_id": "52fe4284c3a36847f8024fa1", "gender": 2, "id": 12899, "name": "Jim Varney", "order": 3, "profile_path": "/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg"}, {"cast_id": 18, "character": "Rex (voice)", "credit_id": "52fe4284c3a36847f8024fa5", "gender": 2, "id": 12900, "name": "Wallace Shawn", "order": 4, "profile_path": "/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg"}, {"cast_id": 19, "character": "Hamm (voice)", "credit_id": "52fe4284c3a36847f8024fa9", "gender": 2, "id": 7907, "name": "John Ratzenberger", "order": 5, "profile_path": "/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg"}, {"cast_id": 20, "character": "Bo Peep (voice)", "credit_id": "52fe4284c3a36847f8024fad", "gender": 1, "id": 8873, "name": "Annie Potts", "order": 6, "profile_path": "/eryXT84RL41jHSJcMy4kS3u9y6w.jpg"}, {"cast_id": 26, "character": "Andy (voice)", "credit_id": "52fe4284c3a36847f8024fc1", "gender": 0, "id": 1116442, "name": "John Morris", "order": 7, "profile_path": "/vYGyvK4LzeaUCoNSHtsu


In [159]:
# 1. import the "pprintjson" function.


# 2. pretty print JSON.
#obj = { "a": 1, "b": "string", "c": True }
obj = {'credit_id': '52fe43b0c3a36847f8067e55', 'department': 'Directing', 'gender': 2, 'id': 2636, 'job': 'Director', 'name': 'Alfred Hitchcock', 'profile_path': '/nA6zRjaVk1JjYvRpuihXJ2pirWk.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e5b', 'department': 'Writing', 'gender': 2, 'id': 4341, 'job': 'Screenplay', 'name': 'Ben Hecht', 'profile_path': '/is5faYjaFObagG3b8xgwocMkUcX.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e61', 'department': 'Production', 'gender': 2, 'id': 3248, 'job': 'Producer', 'name': 'David O. Selznick', 'profile_path': None}, {'credit_id': '52fe43b0c3a36847f8067e67', 'department': 'Sound', 'gender': 2, 'id': 7647, 'job': 'Original Music Composer', 'name': 'Miklós Rózsa', 'profile_path': '/1eAU1ckh7SUNMP9IZeii9qX5xa.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e6d', 'department': 'Art', 'gender': 2, 'id': 2763, 'job': 'Art Direction', 'name': 'James Basevi', 'profile_path': None}, {'credit_id': '52fe43b0c3a36847f8067e73', 'department': 'Camera', 'gender': 2, 'id': 3356, 'job': 'Director of Photography', 'name': 'George Barnes', 'profile_path': '/rSN2fSx7HyorrMdBQ5IyIkskvYW.jpg'}

ppjson(obj)

[
    {
        "credit_id": "52fe43b0c3a36847f8067e55",
        "department": "Directing",
        "gender": 2,
        "id": 2636,
        "job": "Director",
        "name": "Alfred Hitchcock",
        "profile_path": "/nA6zRjaVk1JjYvRpuihXJ2pirWk.jpg"
    },
    {
        "credit_id": "52fe43b0c3a36847f8067e5b",
        "department": "Writing",
        "gender": 2,
        "id": 4341,
        "job": "Screenplay",
        "name": "Ben Hecht",
        "profile_path": "/is5faYjaFObagG3b8xgwocMkUcX.jpg"
    },
    {
        "credit_id": "52fe43b0c3a36847f8067e61",
        "department": "Production",
        "gender": 2,
        "id": 3248,
        "job": "Producer",
        "name": "David O. Selznick",
        "profile_path": null
    },
    {
        "credit_id": "52fe43b0c3a36847f8067e67",
        "department": "Sound",
        "gender": 2,
        "id": 7647,
        "job": "Original Music Composer",
        "name": "Mikl\u00f3s R\u00f3zsa",
        "profile_path": "/1eAU1ckh7SUNMP9I

In [ ]:
import json

def is_json(obj):
    try:
        json.loads(obj)
        return True
    except ValueError:
        return False

# Ejemplo de DataFrame
#df = pd.DataFrame({'columna': ['{"nombre": "John", "edad": 30}', 'No es JSON', '{"nombre": "Jane", "edad": 25}']})

# Aplicar la función is_json a la columna 'columna'
df_moviescredits['comprueba_equipo'] = df_moviescredits['equipo'].apply(lambda x: is_json(x))

print(df_moviescredits.equipo.sample(15))


In [127]:
import pandas as pd
import json

def comprobar_columna(df, columna):
    posiciones_json = []
    posiciones_none = []
    
    for i, valor in enumerate(df[columna]):
        if isinstance(valor, str):
            try:
                json.loads(valor)
                posiciones_json.append(i)
            except ValueError:
                pass
        elif valor is None:
            posiciones_none.append(i)
    
    return posiciones_json, posiciones_none

# Ejemplo de DataFrame
#df = pd.DataFrame({'columna': ['{"nombre": "John", "edad": 30}', 'No es JSON', None, '{"nombre": "Jane", "edad": 25}']})

# Llamada a la función
pos_json, pos_none = comprobar_columna(df_moviescredits, 'actores')

print("Posiciones de objetos JSON:", pos_json)
print("Posiciones de objetos None:", pos_none)


Posiciones de objetos JSON: [189, 614, 635, 661, 861, 1105, 1270, 1391, 1403, 1459, 1487, 1624, 1626, 1630, 1632, 1689, 2369, 2423, 2965, 3106, 3753, 3759, 3941, 4183, 4325, 4329, 4626, 5182, 5481, 5615, 5763, 6051, 6110, 6583, 6604, 6612, 6785, 6987, 7138, 7193, 7268, 8208, 8228, 8243, 8287, 9162, 9293, 9778, 9825, 10238, 10860, 11283, 11386, 11940, 12248, 12333, 13046, 13164, 13165, 13221, 13230, 13500, 13587, 13594, 13618, 13655, 13751, 13754, 13907, 13939, 14061, 14403, 14448, 14530, 14539, 14609, 14685, 14689, 14745, 15028, 15030, 15075, 15242, 15256, 15309, 15369, 15435, 15522, 15612, 15765, 15784, 15850, 15870, 15963, 16422, 16598, 16601, 16610, 16655, 16829, 16833, 16896, 16907, 17095, 17193, 17217, 17316, 17322, 17412, 17416, 17447, 17451, 17492, 17563, 17647, 17673, 17674, 17826, 18025, 18062, 18427, 18497, 18566, 18578, 18602, 18619, 18658, 18696, 18733, 18736, 18791, 18854, 18926, 19019, 19093, 19104, 19170, 19179, 19219, 19299, 19336, 19351, 19353, 19358, 19398, 19400, 194

In [124]:
print(df_moviescredits.equipo.iloc[19714])

None


In [147]:
import pandas as pd
import json

def comprobar_columna(df, columna):
    posiciones_json = []
    posiciones_none = []

    for i, valor in enumerate(df[columna]):
        if isinstance(valor, str):
            try:
                json.loads(valor)
                posiciones_json.append(i)
            except ValueError:
                pass
        elif valor is None:
            posiciones_none.append(i)
    
    return posiciones_json, posiciones_none

# Ejemplo de DataFrame
#df = pd.DataFrame({'columna': ['{"nombre": "John", "edad": 30}', 'No es JSON', None, '{"nombre": "Jane", "edad": 25}']})

# Llamada a la función
pos_json, pos_none = comprobar_columna(df_myc, 'actores')

print("Posiciones de objetos JSON:", pos_json)
print("Posiciones de objetos None:", pos_none)


Posiciones de objetos JSON: []
Posiciones de objetos None: []


In [148]:
import pandas as pd
import json

def comprobar_columna(df, columna):
    posiciones_json = []
    posiciones_none = []

    for i, valor in enumerate(df[columna]):
        if isinstance(valor, str):
            try:
                json.loads(valor)
                posiciones_json.append(i)
            except ValueError:
                pass
        elif valor is None:
            posiciones_none.append(i)
    
    return posiciones_json, posiciones_none

# Llamada a la función
pos_json, pos_none = comprobar_columna(df_myc, 'equipo')

print("Posiciones de objetos JSON:", pos_json)
print("Posiciones de objetos None:", pos_none)


Posiciones de objetos JSON: []
Posiciones de objetos None: []


In [31]:
import openpyxl
df_test = pd.read_excel('D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\dftest_160623.xlsx', sheet_name='Sheet1')

In [7]:
df_test.sample(2)

,id,actores,imdb_id,title,equipo,belongs_to_collection,genres,original_language,overview,popularity,...,release_day,runtime,spoken_languages,status,tagline,vote_average,vote_count,budget,revenue,return
18338,75301,"{""cast_id"": 2, ""character"": ""Himself"", ""credit...",tt1787660,"Being Elmo: A Puppeteer""s Journey","{""credit_id"": ""52fe48fbc3a368484e114981"", ""dep...","{""id"": 999999, ""name"": ""Sin dato""}","{""id"": 99, ""name"": ""Documentary""}, {""id"": 1075...",en,Beloved by children of all ages around the wor...,6.948810,...,21,80,"{""iso_639_1"": ""en"", ""name"": ""English""}",Released,Sin dato,7.4,62,0,286201,0.0
26244,54940,"{""cast_id"": 1, ""character"": ""Jack Harris"", ""cr...",tt0137799,Ground Control,"{""credit_id"": ""533823b19251417da7002aa2"", ""dep...","{""id"": 999999, ""name"": ""Sin dato""}","{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name...",en,An air-traffic controller quits after a plane ...,8.437586,...,26,98,"{""iso_639_1"": ""en"", ""name"": ""English""}",Released,No Radar. No Contact. No Control.,5.6,18,500000,0,0.0


In [226]:
# Guardamos los datos en un CSV

df_test.to_csv(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\dftest_160623.csv', index= False, encoding='utf-8')

In [181]:
import pandas as pd
import json

def comprobar_columna(df, columna):
    posiciones_json = []
    posiciones_str  = []
    posiciones_bool = []

    for i, valor in enumerate(df[columna]):
        if isinstance(valor, str):
            try:
                json.loads(valor)
                posiciones_json.append(i)
            except ValueError:
                posiciones_str.append(i)
        elif isinstance(valor, bool):
            posiciones_bool.append(i)
    
    return posiciones_json, posiciones_str, posiciones_bool

# Ejemplo de DataFrame
#df = pd.DataFrame({'columna': ['{"nombre": "John", "edad": 30}', 'No es JSON', True, '{"nombre": "Jane", "edad": 25}']})

# Llamada a la función
pos_json, pos_str, pos_bool = comprobar_columna(df_myc, 'production_companies')

print("Posiciones de objetos JSON:", pos_json)
print(f'son {len(pos_json)} registros')
print("Posiciones de objetos string:", pos_str)
print(f'son {len(pos_str)} registros')
print("Posiciones de objetos booleanos:", pos_bool)
print(f'son {len(pos_bool)} registros')


Posiciones de objetos JSON: [0, 3, 7, 19, 26, 27, 32, 36, 38, 42, 48, 50, 52, 57, 58, 60, 65, 66, 68, 77, 78, 80, 83, 87, 88, 91, 96, 100, 104, 106, 107, 114, 115, 117, 118, 119, 120, 122, 123, 126, 127, 128, 130, 132, 133, 134, 135, 137, 139, 140, 141, 143, 154, 159, 171, 172, 176, 181, 182, 184, 185, 189, 191, 193, 195, 198, 202, 203, 207, 208, 209, 213, 214, 217, 223, 230, 231, 234, 235, 236, 237, 239, 240, 241, 248, 249, 255, 257, 258, 260, 263, 266, 267, 270, 271, 273, 274, 279, 281, 283, 285, 290, 291, 294, 299, 301, 305, 306, 308, 309, 316, 319, 322, 324, 326, 327, 328, 329, 330, 332, 333, 335, 340, 341, 343, 346, 351, 356, 357, 360, 361, 365, 370, 372, 373, 376, 377, 384, 385, 386, 390, 394, 396, 398, 399, 401, 402, 403, 404, 406, 407, 410, 414, 416, 417, 419, 422, 424, 428, 430, 432, 438, 439, 440, 444, 447, 451, 452, 453, 455, 457, 458, 465, 468, 471, 472, 477, 478, 479, 481, 482, 487, 488, 497, 505, 506, 507, 508, 514, 519, 524, 528, 531, 534, 537, 538, 540, 542, 544, 551, 5

In [ ]:
import json

def validar_json(json_str):
    try:
        json.loads(json_str)
        return True
    except json.JSONDecodeError:
        return False

# Ejemplo de validación de un objeto JSON
objeto_json = '{"nombre": "John", "edad": 30, "trabajo": "programador"}'
es_valido = validar_json(objeto_json)

if es_valido:
    print("El objeto JSON tiene formato y sintaxis válida")
else:
    print("El objeto JSON tiene errores de formato o sintaxis")


In [182]:
import pandas as pd
import json

def comprobar_json(df, columna):
    posiciones_json = []
    revisar_json  = []
    otro_error    = []
    

    for i, valor in enumerate(df[columna]):
        if isinstance(valor, str):
            try:
                json.loads(valor)
                posiciones_json.append(i)
            except ValueError:
                revisar_json.append(i)
        elif isinstance(valor, bool):
            otro_error.append(i)
    
    return posiciones_json, revisar_json, otro_error

# Llamada a la función
pos_json, pos_nojson, pos_otros = comprobar_json(df_myc, 'production_companies')

print("Posiciones de objetos JSON:", pos_json)
print("Posiciones de objetos string:", pos_str)
print("Largo de objetos string:", len(pos_str))
print("Posiciones de objetos booleanos:", pos_bool)



Posiciones de objetos JSON: [0, 3, 7, 19, 26, 27, 32, 36, 38, 42, 48, 50, 52, 57, 58, 60, 65, 66, 68, 77, 78, 80, 83, 87, 88, 91, 96, 100, 104, 106, 107, 114, 115, 117, 118, 119, 120, 122, 123, 126, 127, 128, 130, 132, 133, 134, 135, 137, 139, 140, 141, 143, 154, 159, 171, 172, 176, 181, 182, 184, 185, 189, 191, 193, 195, 198, 202, 203, 207, 208, 209, 213, 214, 217, 223, 230, 231, 234, 235, 236, 237, 239, 240, 241, 248, 249, 255, 257, 258, 260, 263, 266, 267, 270, 271, 273, 274, 279, 281, 283, 285, 290, 291, 294, 299, 301, 305, 306, 308, 309, 316, 319, 322, 324, 326, 327, 328, 329, 330, 332, 333, 335, 340, 341, 343, 346, 351, 356, 357, 360, 361, 365, 370, 372, 373, 376, 377, 384, 385, 386, 390, 394, 396, 398, 399, 401, 402, 403, 404, 406, 407, 410, 414, 416, 417, 419, 422, 424, 428, 430, 432, 438, 439, 440, 444, 447, 451, 452, 453, 455, 457, 458, 465, 468, 471, 472, 477, 478, 479, 481, 482, 487, 488, 497, 505, 506, 507, 508, 514, 519, 524, 528, 531, 534, 537, 538, 540, 542, 544, 551, 5

In [156]:
df_myc.belongs_to_collection.iloc[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [183]:
df_myc.head(1)

,id,imdb_id,title,belongs_to_collection,genres,original_language,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,vote_average,vote_count,budget,revenue,return,equipo,actores
0,862,tt0114709,Toy Story,"{""id"": 10194, ""name"": ""Toy Story Collection"", ...","{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""n...",en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,"{""name"": ""Pixar Animation Studios"", ""id"": 3}","{""iso_3166_1"": ""US"", ""name"": ""United States of...",...,"{""iso_639_1"": ""en"", ""name"": ""English""}",Released,Sin dato,7.7,5415,30000000,373554033,1245.18011,"{""credit_id"": ""52fe4284c3a36847f8024f49"", ""dep...","{""cast_id"": 14, ""character"": ""Woody (voice)"", ..."


In [ ]:
df_myc.columns

In [ ]:
import json

def convertir_a_json(columna):
    return columna.apply(lambda x: json.loads(x) if isinstance(x, str) else x)

In [ ]:
from pprintjson import pprintjson as ppjson

def imprimir_rango_filas_columna(dataframe, columna, inicio, fin):
    valores_columna = dataframe[columna]
    for valor in valores_columna[inicio:fin]:
        print()
        json.loads(valor)
        ppjson(valor)
        print()

# Ejemplo de uso
df      = df_myc
columna = 'production_companies'
inicio  = 1
fin     = 3
imprimir_rango_filas_columna(df, columna, inicio, fin)


# 2. pretty print JSON.
#obj = { "a": 1, "b": "string", "c": True }
#obj = {'credit_id': '52fe43b0c3a36847f8067e55', 'department': 'Directing', 'gender': 2, 'id': 2636, 'job': 'Director', 'name': 'Alfred Hitchcock', 'profile_path': '/nA6zRjaVk1JjYvRpuihXJ2pirWk.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e5b', 'department': 'Writing', 'gender': 2, 'id': 4341, 'job': 'Screenplay', 'name': 'Ben Hecht', 'profile_path': '/is5faYjaFObagG3b8xgwocMkUcX.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e61', 'department': 'Production', 'gender': 2, 'id': 3248, 'job': 'Producer', 'name': 'David O. Selznick', 'profile_path': None}, {'credit_id': '52fe43b0c3a36847f8067e67', 'department': 'Sound', 'gender': 2, 'id': 7647, 'job': 'Original Music Composer', 'name': 'Miklós Rózsa', 'profile_path': '/1eAU1ckh7SUNMP9IZeii9qX5xa.jpg'}, {'credit_id': '52fe43b0c3a36847f8067e6d', 'department': 'Art', 'gender': 2, 'id': 2763, 'job': 'Art Direction', 'name': 'James Basevi', 'profile_path': None}, {'credit_id': '52fe43b0c3a36847f8067e73', 'department': 'Camera', 'gender': 2, 'id': 3356, 'job': 'Director of Photography', 'name': 'George Barnes', 'profile_path': '/rSN2fSx7HyorrMdBQ5IyIkskvYW.jpg'}

#ppjson(obj)


In [213]:
df_myc['equipo'].replace('"profile_path": None', '"Sin dato"', inplace=True)

In [214]:
df_myc['equipo'][4019]

'{"credit_id": "52fe43c89251416c7501e013", "department": "Directing", "gender": 2, "id": 17494, "job": "Director", "name": "Dennis Dugan", "profile_path": "/A9Hv8qqB4fgb5IMjEESzBo8LXPZ.jpg"}, {"credit_id": "52fe43c89251416c7501e019", "department": "Writing", "gender": 2, "id": 67354, "job": "Screenplay", "name": "Hank Nelken", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e01f", "department": "Production", "gender": 2, "id": 1296, "job": "Producer", "name": "Bruce Berman", "profile_path": "/nXKlbRLV5aKzIW6XViCAiqWbxPk.jpg"}, {"credit_id": "52fe43c89251416c7501e025", "department": "Writing", "gender": 2, "id": 67376, "job": "Screenplay", "name": "Greg DePaul", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e02b", "department": "Camera", "gender": 2, "id": 58187, "job": "Director of Photography", "name": "Arthur Albert", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e031", "department": "Sound", "gender": 2, "id": 69669, "job": "Music", "name": "Neil Di

In [ ]:
obj = [{
	"credit_id": "52fe43c89251416c7501e013",
	"department": "Directing",
	"gender": 2,
	"id": 17494,
	"job": "Director",
	"name": "Dennis Dugan",
	"profile_path": "/A9Hv8qqB4fgb5IMjEESzBo8LXPZ.jpg"
}, {
	"credit_id": "52fe43c89251416c7501e019",
	"department": "Writing",
	"gender": 2,
	"id": 67354,
	"job": "Screenplay",
	"name": "Hank Nelken",
	"profile_path": "Sin dato"
}, {
	"credit_id": "52fe43c89251416c7501e01f",
	"department": "Production",
	"gender": 2,
	"id": 1296,
	"job": "Producer",
	"name": "Bruce Berman",
	"profile_path": "/nXKlbRLV5aKzIW6XViCAiqWbxPk.jpg"
}, {
	"credit_id": "52fe43c89251416c7501e025",
	"department": "Writing",
	"gender": 2,
	"id": 67376,
	"job": "Screenplay",
	"name": "Greg DePaul",
	"profile_path": "Sin dato"
}, {
	"credit_id": "52fe43c89251416c7501e02b",
	"department": "Camera",
	"gender": 2,
	"id": 58187,
	"job": "Director of Photography",
	"name": "Arthur Albert",
	"profile_path": "Sin dato"
}, {
	"credit_id": "52fe43c89251416c7501e031",
	"department": "Sound",
	"gender": 2,
	"id": 69669,
	"job": "Music",
	"name": "Neil Diamond",
	"profile_path": "/zG7RJH6CfCgmtKz66NGlwLN0fJM.jpg"
}, {
	"credit_id": "52fe43c89251416c7501e037",
	"department": "Editing",
	"gender": 0,
	"id": 69670,
	"job": "Editor",
	"name": "Patrick J. Don Vito",
	"profile_path": "Sin dato"
}, {
	"credit_id": "564d1c489251414dcb00871a",
	"department": "Crew",
	"gender": 2,
	"id": 1007395,
	"job": "Stunt Coordinator",
	"name": "J.J. Makaro",
	"profile_path": "Sin dato"
}]
ppjson(obj)

In [218]:
import pandas as pd

# Crear un DataFrame de ejemplo
#data = {'A': [1, 2, None, 4, 5],
#        'B': [None, 6, 7, 8, None],
#        'C': [9, None, 11, None, 13]}
df = df_myc['equipo']

# Rellenar los valores faltantes con un valor específico
df_filled = df.fillna(0)

# Imprimir el DataFrame original
print("DataFrame original:")
print(df_myc['equipo'][4019])

# Imprimir el DataFrame con valores faltantes rellenados
print("\nDataFrame con valores faltantes rellenados:")
print(df_filled[4019])


DataFrame original:
{"credit_id": "52fe43c89251416c7501e013", "department": "Directing", "gender": 2, "id": 17494, "job": "Director", "name": "Dennis Dugan", "profile_path": "/A9Hv8qqB4fgb5IMjEESzBo8LXPZ.jpg"}, {"credit_id": "52fe43c89251416c7501e019", "department": "Writing", "gender": 2, "id": 67354, "job": "Screenplay", "name": "Hank Nelken", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e01f", "department": "Production", "gender": 2, "id": 1296, "job": "Producer", "name": "Bruce Berman", "profile_path": "/nXKlbRLV5aKzIW6XViCAiqWbxPk.jpg"}, {"credit_id": "52fe43c89251416c7501e025", "department": "Writing", "gender": 2, "id": 67376, "job": "Screenplay", "name": "Greg DePaul", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e02b", "department": "Camera", "gender": 2, "id": 58187, "job": "Director of Photography", "name": "Arthur Albert", "profile_path": None}, {"credit_id": "52fe43c89251416c7501e031", "department": "Sound", "gender": 2, "id": 69669, "job": "Music

In [ ]:
import pandas as pd
import json

# Función recursiva para eliminar los valores None en un objeto JSON
def eliminar_none(obj):
    if isinstance(obj, list):
        return [eliminar_none(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: eliminar_none(value) for key, value in obj.items() if value is not None}
    else:
        return obj

# Crear un DataFrame de ejemplo
# data = {'columna_json': [{'id': 1, 'nombre': 'John', 'valores': [1, None, 3]}, None, {'id': 3, 'nombre': 'Alice', 'valores': [1, 2, None]}, {'id': 4, 'nombre': 'Bob', 'valores': [None, 2, 3]}, None]}
# df = pd.DataFrame(data)

# Eliminar los valores None en la columna_json
df_test['columna_json'] = df['columna_json'].apply(lambda x: eliminar_none(x))

# Imprimir el DataFrame original
print("DataFrame original:")
print(df)

# Convertir la columna_json de nuevo a JSON para visualización
df['columna_json'] = df['columna_json'].apply(json.dumps)

# Imprimir el DataFrame con los valores None eliminados
print("\nDataFrame con los valores None eliminados:")
print(df)

In [50]:
df_addcol_test = df_test.copy()

In [222]:
import pandas as pd
import json

# Función recursiva para reemplazar los valores None en un objeto JSON
def reemplazar_none(obj, reemplazo):
    if isinstance(obj, list):
        return [reemplazar_none(item, reemplazo) for item in obj]
    elif isinstance(obj, dict):
        return {key: reemplazar_none(value, reemplazo) if value is not None else reemplazo for key, value in obj.items()}
    else:
        return obj

# Reemplazar los valores None en la columna_json por "Sin Dato"
reemplazo = "Sin Dato"

#columna_json = ['equipo','actores']
#for columna in columna_json:
df_test['actores'] = df_test['actores'].apply(lambda x: reemplazar_none(x, reemplazo))

# Imprimir el DataFrame original
print("DataFrame original:")
print(df_myc.head(2))

# Convertir la columna_json de nuevo a JSON para visualización
#lst_json = ['belongs_to_collection', 'genres', 'production_companies',
#       'production_countries', 'spoken_languages', 'equipo','actores']

#columna_json = ['equipo','actores']

#for columna in lst_json:
#    df['columna_json'] = df['columna_json'].apply(json.dumps)

# Imprimir el DataFrame con los valores None reemplazados
print("\nDataFrame con los valores None reemplazados:")
print(df_test.head(2))

DataFrame original:
     id    imdb_id      title  \
0   862  tt0114709  Toy Story   
1  8844  tt0113497    Jumanji   

                               belongs_to_collection  \
0  {"id": 10194, "name": "Toy Story Collection", ...   
1                 {"id": 999999, "name": "Sin dato"}   

                                              genres original_language  \
0  {"id": 16, "name": "Animation"}, {"id": 35, "n...                en   
1  {"id": 12, "name": "Adventure"}, {"id": 14, "n...                en   

                                            overview  popularity  \
0  Led by Woody, Andy"s toys live happily in his ...   21.946943   
1  When siblings Judy and Peter discover an encha...   17.015539   

                                production_companies  \
0       {"name": "Pixar Animation Studios", "id": 3}   
1  {"name": "TriStar Pictures", "id": 559}, {"nam...   

                                production_countries  ...  \
0  {"iso_3166_1": "US", "name": "United States of... 

In [ ]:
lst_json = ['belongs_to_collection', 'genres', 'production_companies',
            'production_countries',  'spoken_languages', 'equipo','actores']
import json

#def convertir_a_json(columna):
#    return columna.apply(lambda x: json.loads(x) if isinstance(x, str) else x)

for columna in lst_json:
       # Hacer algo con el valor de la columna
       #convertir_a_json(df_myc[columna])
       df_myc[columna].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
       print(f'columna {columna} convertida a json')
       print(f'columna {df_myc[columna][2]} convertida a json')

#columna_json = convertir_a_json(df['mi_columna'])



In [15]:
# Guardamos los datos en un CSV

df_work.to_csv(r'D:\\BootcampHDS\\M07_ProyInd\\PI_DSPT01_0623\\stagedata\\dfwork_200623.csv', index= False, encoding='utf-8')

In [176]:
from datetime import datetime

def f_dia_sem(v_fec):
    dic_sem = {
        1: 'lunes',
        2: 'martes',
        3: 'miércoles',
        4: 'jueves',
        5: 'viernes',
        6: 'sábado',
        7: 'domingo'
    }
    v_fec_tmp = v_fec.isoweekday()
    
    return dic_sem.get(v_fec_tmp, 'desconocido')


In [197]:
print(type(f_dia_sem(df_work.release_date.iloc[39110])))

<class 'str'>


In [ ]:
fecha = datetime.date(2021, 7, 22)

In [155]:
fecha = datetime(2021, 7, 22)
fecha.strftime("%A")
print(fecha)
print(type(fecha))

2021-07-22 00:00:00
<class 'datetime.datetime'>


In [61]:
obtener_dia_semana(df_work.release_date.iloc[39123])

NameError: name 'obtener_dia_semana' is not defined

In [22]:
# Modificción que valida el día de texto a datetime

import pandas as pd
from datetime import datetime

def f_cantidad_filmaciones_dia(v_Dia):
    # Mapea los días de la semana en español a su valor numérico
    dic_map_dias = {
        'lunes'    : 1,
        'martes'   : 2,
        'miércoles': 3,
        'jueves'   : 4,
        'viernes'  : 5,
        'sábado'   : 6,
        'domingo'  : 7
    }

    # Convierte una cadena de texto que representa el día a minúsculas
    # y obtiene su valor numérico
    v_Dia = v_Dia.lower()      # Convierte la cadena a minúsculas

    if v_Dia in dic_map_dias:  # Verifica que la cadena v_Dia esté en el dic  
        v_dia_num = dic_map_dias[v_Dia]
    else:
        raise ValueError('Nombre de día inválido.') 

    # Filtra el DataFrame por el día de la semana
    df_filtered = df_work[df_work['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday() + 1) == v_dia_num]

    # Obtiene el número de películas
    v_suma_pelis = df_filtered.shape[0]
    print(f"{v_suma_pelis} películas fueron estrenadas un {v_Dia}.")
    return 


In [144]:
# TERCER INTENTO - FILMACIONES X DIA
import pandas as pd
from datetime import datetime

def f_cantidad_filmaciones_dia(v_Dia):
    # Mapea los días de la semana en español a su valor numérico
    dic_map_dias = {
        'lunes'    : 1,
        'martes'   : 2,
        'miércoles': 3,
        'jueves'   : 4,
        'viernes'  : 5,
        'sábado'   : 6,
        'domingo'  : 7
    }

    # Convierte una cadena de texto que representa el día a minúsculas
    # y obtiene su valor numérico
    v_Dia = v_Dia.lower()      # Convierte la cadena a minúsculas

    if v_Dia in dic_map_dias:  # Verifica que la cadena v_Dia esté en el dic  
        v_dia_num = dic_map_dias[v_Dia]
    else:
        raise ValueError('Nombre de día inválido.') 

    # Filtra el DataFrame por el día de la semana
    if isinstance(df_work['release_date'].iloc[0], str):
        df_filtered = df_work[df_work['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday() + 1) == v_dia_num]
    else:
        #df_filtered = df_work[df_work['release_date'].apply(lambda x: x.weekday() + 1) == v_dia_num]
        df_filtered = df_work[df_work['release_date'].apply(lambda x: x.strftime("%A)) == v_dia_num]

    # Obtiene el número de películas
    v_suma_pelis = df_filtered.shape[0]
    print(f"{v_suma_pelis} películas fueron estrenadas un {v_Dia}.")
    return

SyntaxError: unterminated string literal (detected at line 31) (1582276331.py, line 31)

In [ ]:
f_cantidad_filmaciones_dia('sábado')

In [36]:
df_work.sample(1)

,id,actores,imdb_id,title,equipo,belongs_to_collection,genres,original_language,overview,popularity,production_companies,production_countries,release_date,release_year,release_month,release_day,runtime,spoken_languages,status,tagline,vote_average,vote_count,budget,revenue,return,director,elenco
39123,373476,"{""cast_id"": 1, ""character"": ""Alice Manning"", ""...",tt4230700,Complete Unknown,"{""credit_id"": ""56a68d92c3a36819d0000743"", ""dep...","{""id"": 999999, ""name"": ""Sin dato""}","{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""nam...",en,Tom’s birthday dinner party is turned upside d...,4.062516,"{""name"": ""Parts and Labor"", ""id"": 6531}, {""nam...","{""iso_3166_1"": ""US"", ""name"": ""United States of...",2016-08-26 00:00:00,2016,8,26,90,"{""iso_639_1"": ""en"", ""name"": ""English""}",Released,You are who you say you are.,4.9,47,0,187145,0.0,Joshua Marston,"Erin Darke, Rachel Weisz, Hansel Tan, FaTye, Z..."


In [149]:
print(type(df_work.release_date.iloc[39123]))
print(df_work.release_date.iloc[39123])

<class 'datetime.datetime'>
2016-08-26 00:00:00


In [129]:
def extract_date2(date_str):
    # Extraer solo la parte de la fecha sin la hora
    fec = datetime(date_str, '%Y-%m-%d %H:%M:%S').dae
    #return (datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')).strptime('%Y-%m-%d')
    #return datetime.strptime(date_parts, '%Y-%m-%d').date()
    return (datetime(fec, '%Y-%m-%d'))#.date()




In [183]:
fecha = datetime.date(df_work.release_date.iloc[39123])
print (type(fecha))

<class 'datetime.date'>


In [130]:
print(extract_date2(df_work.release_date.iloc[39123]))

TypeError: 'datetime.datetime' object cannot be interpreted as an integer

In [190]:
from datetime import datetime

def f_recorta_fecha(v_fec_long):
    #fecha = datetime.date(v_fec_long)
    #return fecha #.isoweekday()
    return datetime.date(v_fec_long)

In [193]:
def f_recorta_fecha(v_fec_long):
    return datetime.date(v_fec_long)

In [194]:
print(type(f_recorta_fecha(df_work.release_date.iloc[39123])))

<class 'datetime.date'>


In [140]:
fecha_corta = f_recorta_fecha('1969-03-14 00:00:00')

In [141]:
dic_map_dias = {
        'lunes'    : 1,
        'martes'   : 2,
        'miércoles': 3,
        'jueves'   : 4,
        'viernes'  : 5,
        'sábado'   : 6,
        'domingo'  : 7
    }
dia_num = dic_map_dias[fecha_corta]
print(dia_num)

KeyError: 5

In [103]:
def f_dia_semana(f_corta):
    fecha = datetime.strptime(fecha_corta, '%Y-%m-%d')
    dia_semana = fecha.strftime('%A')
    return print (dia_semana)

In [107]:
f_dia_semana(fecha_corta)

'viernes'

In [111]:
from datetime import datetime

def f_dia_semana(v_fec_corta):
    v_fecha = datetime.strptime(v_fec_corta, '%Y-%m-%d')
    v_dia_semana = v_fecha.strftime('%A')
    
    # Mapea los días de la semana en inglés a su equivalente en español
    dic_dias_sem = {'Monday':'lunes','Tuesday':'martes','Wednesday':'miércoles',
                        'Thursday':'jueves','Friday':'viernes',
                        'Saturday':'sábado','Sunday':'domingo'
                        }
    
    #dia_sem_esp = dic_dias_sem[v_dia_semana]
    
    #return dia_semana_espanol
    #return dia_sem_esp
    return dic_dias_sem[v_dia_semana]

In [106]:
f_dia_semana(fecha_corta)

'viernes'

In [95]:
print(type(f_dia_semana('2016-08-26')))

<class 'str'>


In [94]:
print(type(df_work['release_date'].iloc[0]))

<class 'datetime.datetime'>


In [ ]:
## CONVERTIR CONSULTA DE FECHA A DIA

In [112]:
v_fec = df_work['release_date'].iloc[0]
v_fec 'viernes'

datetime.datetime(1995, 10, 30, 0, 0)

In [ ]:
for v_fec in df_work['release_date']:
    print (type(v_fec))

In [ ]:
#v_fec = df_work['release_date'].iloc[0]
v_Dia = 'lunes'
v_suma_pelis = 0
for v_fec in df_work['release_date']:
        v_fec = f_recorta_fecha(v_fec)
        if (f_dia_semana(v_fec)) == v_Dia:
            v_suma_pelis += 1

In [209]:
def f_recorta_fecha(v_fec_long):
    return datetime.date(v_fec_long)

In [211]:
print(type(f_recorta_fecha(df_work['release_date'].iloc[0])))

<class 'datetime.date'>


In [ ]:
def f_dia_sem(v_fec):
    dic_sem = {
        1: 'lunes',
        2: 'martes',
        3: 'miércoles',
        4: 'jueves',
        5: 'viernes',
        6: 'sábado',
        7: 'domingo'
    }
    v_fec_tmp = v_fec.isoweekday()
    
    #return dic_sem.get(v_fec_tmp, 'desconocido')
    
    return dic_sem.get(v_fec_tmp, 'desconocido')

In [212]:

f_dia_sem(df_work['release_date'].iloc[0])

'lunes'

In [ ]:
for v_fec in df_work['release_date']:
    print(v_fec)

In [216]:
v_tmp = f_dia_sem(df_work['release_date'].iloc[0])
print(v_tmp=='lunes')

True


## Fx Votos Titulo en Pandas

In [279]:
import pandas as pd

def votos_titulo(v_titulo):
    # Convertir el título a minúsculas
    v_titulo = v_titulo.title()

    # Pipe de la consulta en el df
    query = df_work['title'] == v_titulo
    movie = df_work.loc[query, ['title', 'release_year', 'vote_count', 'vote_average']].head(1)

    # Verifica si encontró la película
    if not movie.empty:
        v_titulo       = movie['title'].iloc[0]
        v_qtty_votos   = movie['vote_count'].iloc[0]
        
        if v_qtty_votos < 2000:
            print("Puntaje insuficiente!!")
        else:
            v_med_votos    = movie['vote_average'].iloc[0]
            v_anio_estreno = movie['release_year'].iloc[0]
            print(f"La película {v_titulo} fue estrenada en el año {int(v_anio_estreno)}, cuenta con {int(v_qtty_votos)} valoraciones, con un promedio de {v_med_votos}")
    else:
        print(f"No se encontró la película {v_titulo}")

# Test

votos_titulo('Grumpier old men')
votos_titulo('Titanic')
votos_titulo('The Terminator')

Puntaje insuficiente!!
La película Titanic fue estrenada en el año 1997, cuenta con 7770 valoraciones, con un promedio de 7.5
La película The Terminator fue estrenada en el año 1984, cuenta con 4208 valoraciones, con un promedio de 7.4


## Prueba de funciones EDA

In [281]:
df_movies_or.columns

Index(['budget', 'id', 'imdb_id', 'original_language', 'overview',
       'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline',
       'title', 'vote_average', 'vote_count', 'release_year', 'release_month',
       'release_day', 'release_weekday', 'return', 'director', 'elenco',
       'generos', 'pais', 'productora', 'critica_cine'],
      dtype='object')

In [282]:
df_movies_or.head(1)

,budget,id,imdb_id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,release_year,release_month,release_day,release_weekday,return,director,elenco,generos,pais,productora,critica_cine
0,30000000.0,862,tt0114709,en,"Led by Woody, Andy""s toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Sin dato,Toy Story,7.7,5415.0,1995.0,10.0,30.0,1.0,12.451801,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Animation, Adventure, Comedy",USA,Pixar Animation Studios,160.0


___


In [ ]:
from matplotlib import pyplot as plt

genres = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 
          'return', 'critica_cine']
final  = []
for row in genres:
    merge_df = df_work.loc[(df_work['generos'] == row) & (df_work['release_year'] == '2019')]
    values   = merge_df['vote_average']
    
    final.append(values)

#Calculate the IQR and Quantitatively determine if there are any potential outliers

    quartiles = values.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr    = upperq-lowerq
    
    print(f"{row}")
    print(f"‾‾‾‾‾‾‾‾‾")
    print(f"The lower quartile of ratings  is: {lowerq}")
    print(f"The upper quartile of ratings is: {upperq}")
    print(f"The interquartile range of ratings is: {iqr}")
    print(f"The the median of ratings is: {quartiles[0.5]} ")

    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    print(f"Values below {lower_bound} could be outliers.")
    print(f"Values above {upper_bound} could be outliers.\n")
    
    
plt.boxplot(final)
plt.xticks([1, 2, 3, 4, 5, 6,7,8], genres)
plt.xlabel('\n Genres')
plt.ylabel('Ratings')
plt.title('Genres as per Ratings')

# Save the figure
plt.savefig("Box Plot of Genres as per Ratings.png")
plt.show()

In [ ]:
def recomedacionPorGenero(nombrePelicula, n_similares):
        n_similares = int(n_similares)
        genres = list(set([genre for genres in self.df_movies["genres"].str.split("|") for genre in genres]))
        genre_matrix = []
        for index, row in self.df_movies.iterrows():
            genre_list = row["genres"].split("|")
            genre_vector = [1 if genre in genre_list else 0 for genre in genres]
            genre_matrix.append(genre_vector)
        genre_matrix = pd.DataFrame(genre_matrix, columns=genres)
        contador = 1
        selected_movie = self.df_movies[self.df_movies["title"] == nombrePelicula]
        selected_movie_index = selected_movie.index[0]
        #sacamos las similitudes de los generos
        similarities = cosine_similarity(genre_matrix[selected_movie_index:selected_movie_index+1], genre_matrix).flatten()

        #las metemos en una tupla y las ordenamos de mayor a menor 
        movie_list = [(index, similarity) for index, similarity in enumerate(similarities)]
        movie_list.sort(key=lambda x: x[1], reverse=True)

        listaSimilar = []
        for i in movie_list[0:n_similares]:
            listaSimilar.append(i)


        #la bandera nos sirve para saltarnos la propia peli que buscamos
        #siempre esta a false y si nos encontramos la peli que estamos buscando la activamos a True
        #si esta en True al finalizar el bucle significa que ha saltado el titulo que buscabamos para no repetirse a si mismo 
        #y por lo tanto hay que añadir uno mas para llegar al numero deseado por el usuario
        listaPeliculasMostrar = []
        bandera=False
        if(n_similares>len(self.df_movies)):
            n_similares=len(self.df_movies)-1
        for movie in movie_list[0:n_similares]:
            if(nombrePelicula != self.df_movies.iloc[movie[0]]["title"]):
                listaPeliculasMostrar.append(self.df_movies.iloc[movie[0]]["title"])
                contador+=1
            else:
                bandera=True
        if(bandera):

            mov=movie_list[n_similares][0]
            listaPeliculasMostrar.append(self.df_movies.iloc[mov]["title"])

        return listaPeliculasMostrar #listaSimilar

___
## Pruebas ML


In [ ]:
#library installation

import nltk
import pandas as pd
import numpy as np
import ast 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#loading the data
movies = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')

#merging
#movies = movies.merge(credits , on = 'title')

# features include genre , id , keyword , title , overview , cast , crew 
movies=movies[['id' , 'overview' , 'tagline','director', 'title', 'generos' , 'elenco' , 'productora' ]]
movies.dropna(inplace = True)

#feature conversions
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj) :
        L.append(i['name'])
    return L

def convert3(obj):
    L= []
    counter = 0 
    for i in ast.literal_eval(obj):
        if counter != 5:
            L.append(i['name'])
            counter +=1
        else :
            break
    return L

def fetch_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director' :
            L.append(i['name'])
            break
    return L

movies['generos']     = movies['generos'].apply(convert)
movies['tagline']     = movies['tagline'].apply(convert)
movies['elenco']      = movies['elenco'].apply(convert3)
movies['director'].apply(fetch_director)
movies['overview']    = movies['overview'].apply(lambda x:x.split())
movies['generos']     = movies['generos'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['tagline']     = movies['tagline'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['elenco']      = movies['elenco'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['director']    = movies['director'].apply(lambda x:[i.replace(" " , "")for i in x])
movies['tags']        = movies['overview'] + movies['tagline'] +movies['generos']+movies['elenco'] +movies['director']
new_dataframe         = movies[['id' , 'title' , 'tags']]
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:" ".join(x))
new_dataframe['tags'] = new_dataframe['tags'].apply(lambda x:x.lower())

#feature  scaling
cv = CountVectorizer(max_features=5000 , stop_words='english')
vectors = cv.fit_transform(new_dataframe['tags']).toarray()
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
new_dataframe['tags'] = new_dataframe['tags'].apply(stem)
similarity            = cosine_similarity(vectors)

#recomenddation function
def recomendacion(pelicula):
    movie_index = new_dataframe[new_dataframe['title']==pelicula].index[0]
    distances   = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)) , reverse = True , key=lambda x:x[1])[1:11] 
    for i in movies_list :
        print(new_dataframe.iloc[i[0]].title)

print(recomendacion('Avatar'))

In [4]:
movies = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')

In [5]:
movies.head(1)

,id,imdb_id,title,pais,release_date,status,overview,tagline,release_year,release_month,release_day,release_weekday,director,elenco,productora,original_language,runtime,revenue,budget,return,vote_average,vote_count,popularity,critica_cine,generos
0,862,tt0114709,Toy Story,USA,1995-10-30 00:00:00,Released,"Led by Woody, Andy""s toys live happily in his ...",Sin dato,1995,10,30,1,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",Pixar Animation Studios,en,81,373554033,30000000,12.451801,7.7,5415,21.946943,160,"Animation, Adventure, Comedy"


In [11]:
movies.generos.iloc[2]

'Comedy, Romance'

In [ ]:
def convert(obj):
        return [i['name'] for i in ast.literal_eval(obj)]

movies['generos'].apply(convert)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

movies=pd.read_csv("tmdb_5000_movies.csv")
data=pd.read_csv("tmdb_5000_credits.csv")

movies=movies.merge(data,on='title')

movies
movies.shape

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

movies.head()

import ast

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

movies.dropna(inplace=True)

movies.isnull().sum()

movies['genres'] = movies['genres'].apply(convert)
movies.head()

movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

movies['cast'][0]

def convert3(text):
    L = []
    count=0
    for i in ast.literal_eval(text):
        if count != 3:
            L.append(i['name']) 
            count+=1
        else:
            break
    return L 

movies['cast'] = movies['cast'].apply(convert3)
movies.head()
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

movies['crew'] = movies['crew'].apply(fetch_director)

movies

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

movies['cast']     = movies['cast'].apply(collapse)
movies['crew']     = movies['crew'].apply(collapse)
movies['genres']   = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies.head()
movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new

new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

vector.shape

new['tags']=new['tags'].apply(lambda x:x.lower())

import nltk
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)  

new['tags']=new['tags'].apply(stem)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vectors=cv.fit_transform(new['tags']).toarray()

cv.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vectors)
similarity[2]

sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

def recomienda(movie):
    movie_index=new[new['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new.iloc[i[0]].title)

recomienda('Avatar')

import pickle

pickle.dump(new,open('movies.pkl','wb'))

pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))

pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
from flask_table import Table, Col
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


name            = Col('Recommendations')
description     = Col('Score')


def recommendation():
    # Carga de datos
    df = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')
    
    df = df[['title', 'generos', 'director', 'elenco', 'overview']]
    
    # discarding the commas between the actors' full names and getting only the first three names
    df['elenco'] = df['elenco'].map(lambda x: x.split(',')[:3])
    
    # putting the genres in a list of words
    df['generos']  = df['generos'].map(lambda x: x.lower().split(','))
    df['director'] = df['director'].map(lambda x: x.split(' '))
    
    # merging together first and last name for each actor and director, so it's considered as one word
    # and there is no mix up between people sharing a first name
    for index, row in df.iterrows():
        row['elenco']   = [x.lower().replace(' ', '') for x in row['elenco']]
        row['director'] = ''.join(row['director']).lower()
    
    # initializing the new column
    df['Key_words'] = ""
    for index, row in df.iterrows():
        plot = row['Plot']
        # instantiating Rake, by default is uses english stopwords from NLTK
        # and discard all punctuation characters
        r = Rake()
        # extracting the words by passing the text
        r.extract_keywords_from_text(plot)
        # getting the dictionary with key words and their scores
        key_words_dict_scores = r.get_word_degrees()
        # assigning the key words to the new column
        row['Key_words'] = list(key_words_dict_scores.keys())

    # dropping the Plot column
    df.drop(columns=['overiew'], inplace=True)
    df.set_index('title', inplace=True)
    df['bag_of_words'] = ''
    columns = df.columns
    for index, row in df.iterrows():
        words = ''
        for col in columns:
            if col != 'director':
                words = words + ' '.join(row[col]) + ' '
            else:
                words = words + row[col] + ' '
        row['bag_of_words'] = words

    df.drop(columns=[col for col in df.columns if col != 'bag_of_words'], inplace=True)
    
    # instantiating and generating the count matrix
    count = CountVectorizer()
    count_matrix = count.fit_transform(df['bag_of_words'])
    
    # creating a Series for the movie titles so they are associated to an ordered numerical
    # list I will use later to match the indexes
    indices = pd.Series(df.index)
    
    # generating the cosine similarity matrix
    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    def recomendacions(peli_referencia):

        recommended_movies = []
        # getting the index of the movie that matches the title
        try:
            idx = indices[indices == title].index[0]
        except:
            res = "This movie in not registered in our database"
            return res, ['0']
        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
        # getting the indexes of the 10 most similar movies
        top_10_indexes = list(score_series.iloc[1:11].index)
        # populating the list with the titles of the best 10 matching movies
        for i in top_10_indexes:
            recommended_movies.append(list(df.index)[i])

        return recommended_movies, score_series[1:11]

    int_features = [str(x) for x in request.form.values()]
    int_features = ''.join(int_features)
    output, score = recommendations(int_features)
    if output == "This movie in not registered in our database":
        return render_template('welcome.html', prediction_text=output)
    else:
        x = [str(i) for i in list(round(score, 3))]

        items = [dict(name=output[0], description=x[0]),
                    dict(name=output[1], description=x[1]),
                    dict(name=output[2], description=x[2]),
                    dict(name=output[3], description=x[3]),
                    dict(name=output[4], description=x[4]),
                    dict(name=output[5], description=x[5]),
                    dict(name=output[6], description=x[6]),
                    dict(name=output[7], description=x[7]),
                    dict(name=output[8], description=x[8]),
                    dict(name=output[9], description=x[9])]

        table = Item(items)
        table.border = True
        return render_template(prediction_text=table)


In [ ]:
from flask_table import Col, Table
from flask import Flask, render_template, request
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rake_nltk import Rake


app = Flask(__name__)

class Item(Table):
    name = Col('Recommendations')
    description = Col('Score')


def load_data():
    # Carga de datos
    df = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')
    
    df = df[['title', 'generos', 'director', 'elenco', 'overview']]
    
    # Eliminar comas y otros elementos en los textos
    df['elenco'] = df['elenco'].str.replace(',', '').str.replace('.', '')
    df['generos'] = df['generos'].str.replace(',', '').str.replace('.', '')
    df['director'] = df['director'].str.replace(',', '').str.replace('.', '')
    
    # Poner los géneros en una lista de palabras
    df['generos'] = df['generos'].str.lower().str.split()
    
    # Fusionar nombres y apellidos de actores y directores
    df['elenco'] = df['elenco'].str.lower().str.replace(' ', '')
    df['director'] = df['director'].str.lower().str.replace(' ', '')
    
    # Inicializar la nueva columna de palabras clave
    df['Key_words'] = ""
    
    # Extraer palabras clave utilizando Rake
    for index, row in df.iterrows():
        r = Rake()
        r.extract_keywords_from_text(row['overview'])
        key_words_dict_scores = r.get_word_degrees()
        row['Key_words'] = list(key_words_dict_scores.keys())
    
    # Eliminar columnas no necesarias
    df.drop(columns=['overview'], inplace=True)
    
    # Crear columna 'bag_of_words' acumulando los tokens
    df['bag_of_words'] = df['generos'] + ' ' + df['director'] + ' ' + df['elenco'] + ' ' + df['Key_words']
    
    # Eliminar columnas distintas de 'bag_of_words'
    df.drop(columns=[col for col in df.columns if col != 'bag_of_words'], inplace=True)
    
    return df


def get_recommendations(peli_referencia, df):
    recommended_movies = []
    
    # Obtener el índice de la película de referencia
    try:
        idx = df[df['title'] == peli_referencia].index[0]
    except IndexError:
        return "This movie is not registered in our database", []
    
    # Crear matriz de conteo
    count = CountVectorizer()
    count_matrix = count.fit_transform(df['bag_of_words'])
    
    # Calcular la similaridad coseno
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    # Obtener los índices de las películas más similares
    scores = list(enumerate(cosine_sim[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_10_indexes = [score[0] for score in scores[1:11]]
    
    # Obtener los títulos de las películas recomendadas
    recommended_movies = list(df.iloc[top_10_indexes]['title'])
    
    return recommended_movies


@app.route('/', methods=['GET', 'POST'])
def recomendacion():
    if request.method == 'POST':
        peli_referencia = request.form.get('peli_referencia')
        
        df = load_data()
        recommendations = get_recommendations(peli_referencia, df)
        
        if recommendations[0] == "This movie is not registered in our database":
            return render_template('welcome.html', prediction_text=recommendations[0])
        else:
            items = []
            for movie in recommendations[:10]:
                items.append({'name': movie, 'description': ''})
                
            table = Item(items)
            table.border = True
            return render_template('result.html', table=table)
    
    return render_template('index.html')


if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rake_nltk import Rake
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from gotstopwords import gotstopwords


stopwords = nltk.corpus.stopwords.words('english')

def load_data():
    # Carga de datos
    df = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')
    
    df = df[['title', 'generos', 'tagline', 'productora', 'director', 'elenco', 'overview']]
    
    # Eliminar comas y otros elementos en los textos
    df['elenco']     = df['elenco'].str.replace(',', '').str.replace('.', '')
    df['generos']    = df['generos'].str.replace(',', '').str.replace('.', '')
    df['director']   = df['director'].str.replace(',', '').str.replace('.', '')
    df['title']      = df['title'].str.replace(',', '').str.replace('.', '')
    df['tagline']    = df['tagline'].str.replace(',', '').str.replace('.', '')
    df['productora'] = df['productora'].str.replace(',', '').str.replace('.', '')
    df['overview']   = df['overview'].str.replace(',', '').str.replace('.', '')
    
    # Poner los géneros en una lista de palabras
    df['generos'] = df['generos'].str.lower().str.split()
    
    # Fusionar nombres y apellidos de actores y directores
    df['elenco']   = df['elenco'].str.lower().str.replace(' ', '')
    df['director'] = df['director'].str.lower().str.replace(' ', '')
    
    # Inicializar la nueva columna de palabras clave
    df['Key_words'] = ""
    
    # Extraer palabras clave utilizando Rake
    for index, row in df.iterrows():
        r                     = Rake()
        r.extract_keywords_from_text(row['overview'])
        key_words_dict_scores = r.get_word_degrees()
        row['Key_words']      = list(key_words_dict_scores.keys())
    
    # Eliminar columnas no necesarias
    df.drop(columns=['overview'], inplace=True)
    
    # Crear columna 'bag_of_words' acumulando los tokens
    #df['bag_of_words'] = df['generos'] + df['director'] + df['elenco'] + df['Key_words']
    df['bag_of_words'] = df['generos'].apply(lambda x: ' '.join(x)) + ' ' + df['director'] + ' ' + df['elenco'] + ' ' + df['Key_words'].apply(lambda x: ' '.join(x))

    #row['director'] = ''.join(row['director']).lower()
    # Eliminar columnas distintas de 'bag_of_words'
    df.drop(columns=[col for col in df.columns if col != ['title','bag_of_words']], inplace=True)
    
    return df


def get_recommendations(peli_referencia, df):
    recommended_movies = []
    
    # Obtener el índice de la película de referencia
    try:
        idx = df[df['title'] == peli_referencia].index[0]
    except IndexError:
        return "This movie is not registered in our database", []
    
    # Crear matriz de conteo
    count        = CountVectorizer()
    count_matrix = count.fit_transform(df['bag_of_words'])
    
    # Calcular la similaridad coseno
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    # Obtener los índices de las películas más similares
    scores         = list(enumerate(cosine_sim[idx]))
    scores         = sorted(scores, key=lambda x: x[1], reverse=True)
    top_10_indexes = [score[0] for score in scores[1:11]]
    
    # Obtener los títulos de las películas recomendadas
    recommended_movies = list(df.iloc[top_10_indexes]['title'])
    
    return recommended_movies

if __name__ == '__main__':
    peli_referencia = input("Ingrese el nombre de una película de referencia: ")
    
    df = load_data()
    recommendations = get_recommendations(peli_referencia, df)
    
    if recommendations[0] == "This movie is not registered in our database":
        print(recommendations[0])
    else:
        print("Recomendaciones:")
        for movie in recommendations:
            print(movie)


# Otro intento

In [ ]:
df = pd.read_excel('D:/BootcampHDS/M07_ProyInd/PI_DSPT01_0623/ML/MLwork290623.xlsx', sheet_name='Sheet1')

In [40]:
stopwords = nltk.corpus.stopwords.words('english')

# Carga de datos
df = df[['title', 'generos', 'tagline', 'productora', 'director', 'elenco', 'overview']]

# Eliminar comas y otros elementos en los textos
df['elenco']     = df['elenco'].str.replace(',', '').str.replace('.', '')
df['generos']    = df['generos'].str.replace(',', '').str.replace('.', '')
df['director']   = df['director'].str.replace(',', '').str.replace('.', '')
df['title']      = df['title'].str.replace(',', '').str.replace('.', '')
df['tagline']    = df['tagline'].str.replace(',', '').str.replace('.', '')
df['productora'] = df['productora'].str.replace(',', '').str.replace('.', '')
df['overview']   = df['overview'].str.replace(',', '').str.replace('.', '')

# Poner los géneros en una lista de palabras
df['generos'] = df['generos'].str.lower().str.split()


C:\Users\Estudio\AppData\Local\Temp\ipykernel_14416\4174813283.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['elenco']     = df['elenco'].str.replace(',', '').str.replace('.', '')
C:\Users\Estudio\AppData\Local\Temp\ipykernel_14416\4174813283.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['generos']    = df['generos'].str.replace(',', '').str.replace('.', '')
C:\Users\Estudio\AppData\Local\Temp\ipykernel_14416\4174813283.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['director']   = df['director'

In [41]:

df.head(2)

,title,generos,tagline,productora,director,elenco,overview
0,Toy Story,"[animation, adventure, comedy]",Sin dato,Pixar Animation Studios,John Lasseter,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,"Led by Woody Andy""s toys live happily in his r..."
1,Jumanji,"[adventure, comedy, family]",Roll the dice and unleash the excitement!,TriStar Pictures,Joe Johnston,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,When siblings Judy and Peter discover an encha...



    


# Fusionar nombres y apellidos de actores y directores
df['elenco']   = df['elenco'].str.lower().str.replace(' ', '')
df['director'] = df['director'].str.lower().str.replace(' ', '')

# Inicializar la nueva columna de palabras clave
df['Key_words'] = ""

# Extraer palabras clave utilizando Rake
for index, row in df.iterrows():
    r                     = Rake()
    r.extract_keywords_from_text(row['overview'])
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words']      = list(key_words_dict_scores.keys())

# Eliminar columnas no necesarias
df.drop(columns=['overview'], inplace=True)

# Crear columna 'bag_of_words' acumulando los tokens
#df['bag_of_words'] = df['generos'] + df['director'] + df['elenco'] + df['Key_words']
df['bag_of_words'] = df['generos'].apply(lambda x: ' '.join(x)) + ' ' + df['director'] + ' ' + df['elenco'] + ' ' + df['Key_words'].apply(lambda x: ' '.join(x))

#row['director'] = ''.join(row['director']).lower()
# Eliminar columnas distintas de 'bag_of_words'
df.drop(columns=[col for col in df.columns if col != ['title','bag_of_words']], inplace=True)

return df

In [31]:
df.shape

(45355, 0)